In [1]:
from tinygrad.densetensor import DenseTensor
from tinygrad.sparsetensor import SparseTensor
import numpy as np

%load_ext autoreload
%autoreload 2

DEVICE:GPU


In [2]:
x_init = np.random.randn(2,6).astype(np.float32)
x2_init = np.random.randn(3).astype(np.float32)
U_init = np.random.randn(3,3).astype(np.float32)
V_init = np.random.randn(3,3).astype(np.float32)
W_init = np.random.randn(6,3).astype(np.float32)
m_init = np.random.randn(1,3).astype(np.float32)

x = DenseTensor(x_init)
W = DenseTensor(W_init)
m = DenseTensor(m_init)
out = x.dot(W).relu()
out = out.logsoftmax()
out = out.mul(m).add(m).sum()
out.backward()

out.cpu().data, x

x2 = DenseTensor(x2_init)#.gpu()
W = SparseTensor(W_init)
out = W.dot(x2).relu().sum()

out.backward()

out.cpu().data, x

In [3]:
import numpy as np
import pyopencl as cl

mf = cl.mem_flags

In [4]:
dim1 = 8
dim2 = 16
dim3 = 5
topkx = 5
topky = 8
topk  = topkx
bs = dim3

np.random.seed(9)

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx,
        properties=cl.command_queue_properties.PROFILING_ENABLE)

sparsity = 0.4

a = np.zeros((dim1,dim2))
b = np.zeros((dim2,dim3)).astype(np.float32)

a.shape, b.shape

((8, 16), (16, 5))

In [5]:
x_init = np.random.randn(dim1,dim3).astype(np.float32)
w_init = np.random.randn(dim2,dim3).astype(np.float32)

In [6]:
w_init

array([[-6.56452596e-01, -5.62572964e-02, -4.99902606e-01,
         4.36419368e-01, -3.75813037e-01],
       [-9.23061609e-01,  1.91725028e+00, -1.50302842e-01,
        -6.38729751e-01,  8.24770331e-01],
       [-1.21083879e+00, -5.03405392e-01, -7.01915681e-01,
        -1.97427106e+00, -2.65573215e+00],
       [-5.76822497e-02, -6.56186581e-01, -6.61706686e-01,
         7.69348443e-01, -8.99004877e-01],
       [ 1.69363797e+00, -1.69733524e+00, -2.79337025e+00,
        -2.26150647e-01,  3.97428840e-01],
       [ 1.65970361e+00, -4.93746817e-01, -3.76097679e-01,
        -1.69739768e-01,  2.41710639e+00],
       [-1.80884051e+00,  3.39751154e-01, -2.27297600e-02,
        -9.59997058e-01, -3.83114427e-01],
       [ 1.09529994e-01, -8.55162859e-01,  2.21606664e-04,
         6.63855076e-01,  7.49480963e-01],
       [-4.65818375e-01, -2.77439266e-01,  3.54995355e-02,
         8.48221183e-01,  1.62998557e-01],
       [ 1.20862365e+00,  5.02520800e-01, -1.58382213e+00,
         1.02303350e+00

In [7]:
def fill_sparse(mat, sparsity=0.5):
    indices = np.array(range(mat.shape[1]))
    nrows = int(mat.shape[1]*sparsity)
    for row in range(mat.shape[0]):
        lim = nrows #+ int(np.random.random()*3)
        mat[row][np.random.permutation(indices)[:lim]] = np.random.random(lim)
    return mat

a = fill_sparse(a, sparsity).astype(np.float32)
b = fill_sparse(b, sparsity).astype(np.float32)

In [8]:
a

array([[0.        , 0.68347037, 0.8035886 , 0.        , 0.        ,
        0.        , 0.        , 0.08023349, 0.8858316 , 0.        ,
        0.17861794, 0.        , 0.46201044, 0.        , 0.        ,
        0.        ],
       [0.9562663 , 0.5531271 , 0.        , 0.        , 0.5375557 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.20370705, 0.35530138, 0.        , 0.        ,
        0.9186601 ],
       [0.        , 0.20103766, 0.        , 0.        , 0.        ,
        0.74341315, 0.        , 0.        , 0.        , 0.5787496 ,
        0.        , 0.05387774, 0.        , 0.5917517 , 0.17797045,
        0.        ],
       [0.        , 0.        , 0.        , 0.14296253, 0.9103574 ,
        0.03242496, 0.        , 0.71459514, 0.        , 0.        ,
        0.        , 0.        , 0.7448136 , 0.        , 0.        ,
        0.66635793],
       [0.        , 0.        , 0.        , 0.59893256, 0.        ,
        0.        , 0.        , 

In [9]:
b

array([[0.49395892, 0.        , 0.46451807, 0.        , 0.        ],
       [0.65864426, 0.        , 0.9786625 , 0.        , 0.        ],
       [0.31044865, 0.        , 0.02502257, 0.        , 0.        ],
       [0.        , 0.46765924, 0.        , 0.        , 0.03160795],
       [0.71082693, 0.        , 0.58997136, 0.        , 0.        ],
       [0.        , 0.        , 0.84622455, 0.5916316 , 0.        ],
       [0.46450275, 0.        , 0.        , 0.        , 0.9429863 ],
       [0.        , 0.34248227, 0.7672639 , 0.        , 0.        ],
       [0.        , 0.        , 0.9855889 , 0.        , 0.2535647 ],
       [0.        , 0.7705159 , 0.31522992, 0.        , 0.        ],
       [0.        , 0.7167741 , 0.        , 0.8238369 , 0.        ],
       [0.        , 0.91970533, 0.        , 0.7873889 , 0.        ],
       [0.        , 0.        , 0.        , 0.404568  , 0.841323  ],
       [0.        , 0.        , 0.674503  , 0.48820347, 0.        ],
       [0.18725161, 0.11607183, 0.

In [10]:
x2_init.T

array([ 0.58808726, -0.34174612, -0.9533304 ], dtype=float32)

In [11]:
mult = a.dot(b)
mult.shape

(8, 5)

In [12]:
mult.shape

(8, 5)

In [13]:
def to_data(mat):
    all_rows = []
    all_idxs = []
    all_nnzs = []
    for row in range(mat.shape[0]):
        rowdata = []
        colidxs = []
        all_nnzs.append(0)
        for col in range(mat.shape[1]):
            val = mat[row][col]
            if val != 0:
                rowdata.append(val)
                colidxs.append(col)
                all_nnzs[-1] += 1
        all_rows.append(rowdata)
        all_idxs.append(colidxs)
    
    ellwidth = min(int(np.sqrt(np.max(all_nnzs))+1)**2, mat.shape[1])
    ellwidth = mat.shape[1]
    #all_rows = np.array(all_rows)#.astype(np.float32).flatten()           
    for row in range(mat.shape[0]):
        #print(row, all_rows)
        all_rows[row] = np.array(all_rows[row])
        all_rows[row].resize(ellwidth)
        all_idxs[row] = np.array(all_idxs[row])
        all_idxs[row].resize(ellwidth)
        #print(all_idxs[row])
    all_rows = np.array(all_rows)
    all_idxs = np.array(all_idxs)
    all_nnzs = np.array(all_nnzs)
    
#     while (not all_rows[:,-1].any()):
#         all_rows = all_rows[:,:-1]
#         all_idxs = all_idxs[:,:-1]
#         ellwidth -= 1
        
    
    all_rows = np.array(all_rows).astype(np.float32).flatten()
    all_idxs = np.array(all_idxs).astype(np.uint32).flatten()
    
    all_nnzs = np.array(all_nnzs).astype(np.uint32)
    
    
    return all_rows, all_idxs, all_nnzs, ellwidth

In [14]:
def to_dense(data, cols, nnzs, ellw, shape):
    out = np.zeros(shape)
    for row in range(shape[0]):
        for icol in range(nnzs[row]):
            out[row,cols[row*ellw+icol]] = data[row*ellw+icol]
    return out

In [15]:
wdata, wcols, wnnz, ellww = to_data(w_init)
wdata, wcols, wnnz, ellww

(array([-6.56452596e-01, -5.62572964e-02, -4.99902606e-01,  4.36419368e-01,
        -3.75813037e-01, -9.23061609e-01,  1.91725028e+00, -1.50302842e-01,
        -6.38729751e-01,  8.24770331e-01, -1.21083879e+00, -5.03405392e-01,
        -7.01915681e-01, -1.97427106e+00, -2.65573215e+00, -5.76822497e-02,
        -6.56186581e-01, -6.61706686e-01,  7.69348443e-01, -8.99004877e-01,
         1.69363797e+00, -1.69733524e+00, -2.79337025e+00, -2.26150647e-01,
         3.97428840e-01,  1.65970361e+00, -4.93746817e-01, -3.76097679e-01,
        -1.69739768e-01,  2.41710639e+00, -1.80884051e+00,  3.39751154e-01,
        -2.27297600e-02, -9.59997058e-01, -3.83114427e-01,  1.09529994e-01,
        -8.55162859e-01,  2.21606664e-04,  6.63855076e-01,  7.49480963e-01,
        -4.65818375e-01, -2.77439266e-01,  3.54995355e-02,  8.48221183e-01,
         1.62998557e-01,  1.20862365e+00,  5.02520800e-01, -1.58382213e+00,
         1.02303350e+00, -6.53017402e-01,  5.37045121e-01, -7.97706190e-03,
         9.2

In [16]:
wdatat, wcolst, wnnzt, ellwwt = to_data(w_init.T)
wdatat, wcolst, wnnzt, ellwwt

(array([-6.56452596e-01, -9.23061609e-01, -1.21083879e+00, -5.76822497e-02,
         1.69363797e+00,  1.65970361e+00, -1.80884051e+00,  1.09529994e-01,
        -4.65818375e-01,  1.20862365e+00,  5.37045121e-01, -1.65366137e+00,
         5.61277032e-01, -1.27321005e+00,  9.70861197e-01, -5.30199170e-01,
        -5.62572964e-02,  1.91725028e+00, -5.03405392e-01, -6.56186581e-01,
        -1.69733524e+00, -4.93746817e-01,  3.39751154e-01, -8.55162859e-01,
        -2.77439266e-01,  5.02520800e-01, -7.97706190e-03,  1.36799216e+00,
         2.39725500e-01,  9.20076132e-01,  1.15472412e+00,  8.14134836e-01,
        -4.99902606e-01, -1.50302842e-01, -7.01915681e-01, -6.61706686e-01,
        -2.79337025e+00, -3.76097679e-01, -2.27297600e-02,  2.21606664e-04,
         3.54995355e-02, -1.58382213e+00,  9.24784184e-01,  2.51062457e-02,
         4.79899108e-01,  1.05491853e+00,  4.22608823e-01, -1.01988423e+00,
         4.36419368e-01, -6.38729751e-01, -1.97427106e+00,  7.69348443e-01,
        -2.2

In [17]:
adata, acols, annz, ellwa = to_data(a)
adata, acols, annz, ellwa

(array([0.68347037, 0.8035886 , 0.08023349, 0.8858316 , 0.17861794,
        0.46201044, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.9562663 , 0.5531271 , 0.5375557 , 0.20370705,
        0.35530138, 0.9186601 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.20103766, 0.74341315, 0.5787496 ,
        0.05387774, 0.5917517 , 0.17797045, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.14296253, 0.9103574 ,
        0.03242496, 0.71459514, 0.7448136 , 0.66635793, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.59893256,
        0.9217044 , 0.16333483, 0.17167741, 0.7605819 , 0.48675168,
        0.        , 0.        , 0.        , 0.  

In [18]:
adatat, acolst, annzt, ellwat = to_data(a.T)
adatat, acolst, annzt, ellwat

(array([0.9562663 , 0.11415514, 0.28872353, 0.67626965, 0.        ,
        0.        , 0.        , 0.        , 0.68347037, 0.5531271 ,
        0.20103766, 0.82574075, 0.04889954, 0.        , 0.        ,
        0.        , 0.8035886 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.14296253,
        0.59893256, 0.6493426 , 0.35563806, 0.        , 0.        ,
        0.        , 0.        , 0.5375557 , 0.9103574 , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.74341315, 0.03242496, 0.27978534, 0.33329687, 0.        ,
        0.        , 0.        , 0.        , 0.69630474, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.08023349, 0.71459514, 0.9217044 , 0.25018668,
        0.        , 0.        , 0.        , 0.        , 0.8858316 ,
        0.6753613 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.5787496 , 0.16

In [19]:
bdata, bcols, bnnz, ellwb = to_data(b)
bdata, bcols, bnnz, ellwb

(array([0.49395892, 0.46451807, 0.        , 0.        , 0.        ,
        0.65864426, 0.9786625 , 0.        , 0.        , 0.        ,
        0.31044865, 0.02502257, 0.        , 0.        , 0.        ,
        0.46765924, 0.03160795, 0.        , 0.        , 0.        ,
        0.71082693, 0.58997136, 0.        , 0.        , 0.        ,
        0.84622455, 0.5916316 , 0.        , 0.        , 0.        ,
        0.46450275, 0.9429863 , 0.        , 0.        , 0.        ,
        0.34248227, 0.7672639 , 0.        , 0.        , 0.        ,
        0.9855889 , 0.2535647 , 0.        , 0.        , 0.        ,
        0.7705159 , 0.31522992, 0.        , 0.        , 0.        ,
        0.7167741 , 0.8238369 , 0.        , 0.        , 0.        ,
        0.91970533, 0.7873889 , 0.        , 0.        , 0.        ,
        0.404568  , 0.841323  , 0.        , 0.        , 0.        ,
        0.674503  , 0.48820347, 0.        , 0.        , 0.        ,
        0.18725161, 0.11607183, 0.        , 0.  

In [20]:
bdatat, bcolst, bnnzt, ellwbt = to_data(b.T)
adatat, bcolst, bnnzt, ellwbt

(array([0.9562663 , 0.11415514, 0.28872353, 0.67626965, 0.        ,
        0.        , 0.        , 0.        , 0.68347037, 0.5531271 ,
        0.20103766, 0.82574075, 0.04889954, 0.        , 0.        ,
        0.        , 0.8035886 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.14296253,
        0.59893256, 0.6493426 , 0.35563806, 0.        , 0.        ,
        0.        , 0.        , 0.5375557 , 0.9103574 , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.74341315, 0.03242496, 0.27978534, 0.33329687, 0.        ,
        0.        , 0.        , 0.        , 0.69630474, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.08023349, 0.71459514, 0.9217044 , 0.25018668,
        0.        , 0.        , 0.        , 0.        , 0.8858316 ,
        0.6753613 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.5787496 , 0.16

In [21]:
adense = to_dense(adata, acols, annz, ellwa, a.shape)

In [22]:
adenset = to_dense(adatat, acolst, annzt, ellwat, a.T.shape)

In [23]:
bdense = to_dense(bdata, bcols, bnnz, ellwb, b.shape)

In [24]:
bdenset = to_dense(bdatat, bcolst, bnnzt, ellwbt, b.T.shape)

In [25]:
adense

array([[0.        , 0.68347037, 0.80358863, 0.        , 0.        ,
        0.        , 0.        , 0.08023349, 0.88583159, 0.        ,
        0.17861794, 0.        , 0.46201044, 0.        , 0.        ,
        0.        ],
       [0.95626628, 0.55312711, 0.        , 0.        , 0.53755569,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.20370705, 0.35530138, 0.        , 0.        ,
        0.9186601 ],
       [0.        , 0.20103766, 0.        , 0.        , 0.        ,
        0.74341315, 0.        , 0.        , 0.        , 0.5787496 ,
        0.        , 0.05387774, 0.        , 0.59175169, 0.17797045,
        0.        ],
       [0.        , 0.        , 0.        , 0.14296253, 0.91035742,
        0.03242496, 0.        , 0.71459514, 0.        , 0.        ,
        0.        , 0.        , 0.74481362, 0.        , 0.        ,
        0.66635793],
       [0.        , 0.        , 0.        , 0.59893256, 0.        ,
        0.        , 0.        , 

In [26]:
adenset.T == adense

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  Tru

In [27]:
bdenset.T == bdense

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

In [28]:
a

array([[0.        , 0.68347037, 0.8035886 , 0.        , 0.        ,
        0.        , 0.        , 0.08023349, 0.8858316 , 0.        ,
        0.17861794, 0.        , 0.46201044, 0.        , 0.        ,
        0.        ],
       [0.9562663 , 0.5531271 , 0.        , 0.        , 0.5375557 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.20370705, 0.35530138, 0.        , 0.        ,
        0.9186601 ],
       [0.        , 0.20103766, 0.        , 0.        , 0.        ,
        0.74341315, 0.        , 0.        , 0.        , 0.5787496 ,
        0.        , 0.05387774, 0.        , 0.5917517 , 0.17797045,
        0.        ],
       [0.        , 0.        , 0.        , 0.14296253, 0.9103574 ,
        0.03242496, 0.        , 0.71459514, 0.        , 0.        ,
        0.        , 0.        , 0.7448136 , 0.        , 0.        ,
        0.66635793],
       [0.        , 0.        , 0.        , 0.59893256, 0.        ,
        0.        , 0.        , 

In [29]:
a == adense

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  Tru

In [30]:
a.shape

(8, 16)

In [31]:
adata.shape, acols.shape, annz.shape, ellwa

((128,), (128,), (8,), 16)

In [32]:
#acols = acols.astype(np.uint32)
#annz = annz.astype(np.uint32)

In [33]:
adata, acols, annz, b

(array([0.68347037, 0.8035886 , 0.08023349, 0.8858316 , 0.17861794,
        0.46201044, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.9562663 , 0.5531271 , 0.5375557 , 0.20370705,
        0.35530138, 0.9186601 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.20103766, 0.74341315, 0.5787496 ,
        0.05387774, 0.5917517 , 0.17797045, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.14296253, 0.9103574 ,
        0.03242496, 0.71459514, 0.7448136 , 0.66635793, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.59893256,
        0.9217044 , 0.16333483, 0.17167741, 0.7605819 , 0.48675168,
        0.        , 0.        , 0.        , 0.  

## MatMul (Sparse-Dense)

adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // SPARSE x DENSE
    __kernel void matmul2(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);
      

      uint nnz    = rowNnz[gid];
      float sum = 0;
      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        for (uint i = 0; i < nnz; i++) {
          uint index   = (gid * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          float xval  = vector_x[col*ncols+gid2];
          //if (gid==0 && gid2==2)
          //  printf("aval, xval: %.2f,%.2f: (%i,%i) \\n", aval, xval, col, index);
          sum  += aval * xval;
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid*ncols+gid2] = sum;
      }
    }""").build()

In [34]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // SPARSE x DENSE
    __kernel void matmul2(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);
      

      uint nnz    = rowNnz[gid];
      
      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        float sum = 0;
        for (uint i = 0; i < nnz; i++) {
          uint index   = (gid * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          uint xidx = col*ncols+gid2;
          float xval  = vector_x[xidx];
          if (gid==0 && gid2==1)
            printf("aval, xval: %.2f,%.2f: (%i,%i) - %i \\n", aval, xval, col, index, xidx);
          sum  += aval * xval;
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid*ncols+gid2] = sum;
      }
    }""").build()

In [35]:
a.shape, b.shape

((8, 16), (16, 5))

In [36]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [37]:
rows = a.shape[0]

In [38]:
mult = mult.astype(np.float32)

In [39]:
outshape = (a.shape[0], b.shape[1])
outshape

(8, 5)

In [40]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod(outshape)*4)
knl = prg.matmul2  # Use this Kernel object for repeated calls
knl(queue, [outshape[0]], None, adata_buf, acols_buf, annzs_buf, np.uint32(ellwa), np.uint32(outshape[1]), b_buf, res_buf)

res_np = np.zeros(outshape).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)

aval, xval: 0.68,0.00: (1,0) - 6 
aval, xval: 0.80,0.00: (2,1) - 11 
aval, xval: 0.08,0.34: (7,2) - 36 
aval, xval: 0.89,0.00: (8,3) - 41 
aval, xval: 0.18,0.72: (10,4) - 51 
aval, xval: 0.46,0.00: (12,5) - 61 


In [41]:
(res_np-mult).sum()

0.0

In [42]:
a

array([[0.        , 0.68347037, 0.8035886 , 0.        , 0.        ,
        0.        , 0.        , 0.08023349, 0.8858316 , 0.        ,
        0.17861794, 0.        , 0.46201044, 0.        , 0.        ,
        0.        ],
       [0.9562663 , 0.5531271 , 0.        , 0.        , 0.5375557 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.20370705, 0.35530138, 0.        , 0.        ,
        0.9186601 ],
       [0.        , 0.20103766, 0.        , 0.        , 0.        ,
        0.74341315, 0.        , 0.        , 0.        , 0.5787496 ,
        0.        , 0.05387774, 0.        , 0.5917517 , 0.17797045,
        0.        ],
       [0.        , 0.        , 0.        , 0.14296253, 0.9103574 ,
        0.03242496, 0.        , 0.71459514, 0.        , 0.        ,
        0.        , 0.        , 0.7448136 , 0.        , 0.        ,
        0.66635793],
       [0.        , 0.        , 0.        , 0.59893256, 0.        ,
        0.        , 0.        , 

In [43]:
b

array([[0.49395892, 0.        , 0.46451807, 0.        , 0.        ],
       [0.65864426, 0.        , 0.9786625 , 0.        , 0.        ],
       [0.31044865, 0.        , 0.02502257, 0.        , 0.        ],
       [0.        , 0.46765924, 0.        , 0.        , 0.03160795],
       [0.71082693, 0.        , 0.58997136, 0.        , 0.        ],
       [0.        , 0.        , 0.84622455, 0.5916316 , 0.        ],
       [0.46450275, 0.        , 0.        , 0.        , 0.9429863 ],
       [0.        , 0.34248227, 0.7672639 , 0.        , 0.        ],
       [0.        , 0.        , 0.9855889 , 0.        , 0.2535647 ],
       [0.        , 0.7705159 , 0.31522992, 0.        , 0.        ],
       [0.        , 0.7167741 , 0.        , 0.8238369 , 0.        ],
       [0.        , 0.91970533, 0.        , 0.7873889 , 0.        ],
       [0.        , 0.        , 0.        , 0.404568  , 0.841323  ],
       [0.        , 0.        , 0.674503  , 0.48820347, 0.        ],
       [0.18725161, 0.11607183, 0.

In [44]:
res_buf

In [45]:
res_np

array([[0.6996368 , 0.15550727, 1.6236207 , 0.3340667 , 0.61331564],
       [1.8848312 , 0.18735047, 1.4160932 , 0.30414024, 0.29892322],
       [0.16573755, 0.51614475, 1.40742   , 0.7711447 , 0.        ],
       [1.130233  , 0.31159392, 1.1950791 , 0.32051137, 0.6311476 ],
       [0.49532753, 0.9677906 , 0.8187757 , 0.1351769 , 0.01893103],
       [0.60025734, 0.7728439 , 1.2898692 , 0.9285497 , 0.        ],
       [0.17482497, 0.30367106, 0.47157714, 0.544282  , 0.7309462 ],
       [0.7240762 , 0.27932334, 0.99110806, 0.09674796, 0.8390946 ]],
      dtype=float32)

In [46]:
mult

array([[0.6996368 , 0.15550727, 1.6236207 , 0.3340667 , 0.61331564],
       [1.8848312 , 0.18735047, 1.4160932 , 0.30414024, 0.29892322],
       [0.16573755, 0.51614475, 1.40742   , 0.7711447 , 0.        ],
       [1.130233  , 0.31159392, 1.1950791 , 0.32051137, 0.6311476 ],
       [0.49532753, 0.9677906 , 0.8187757 , 0.1351769 , 0.01893103],
       [0.60025734, 0.7728439 , 1.2898692 , 0.9285497 , 0.        ],
       [0.17482497, 0.30367106, 0.47157714, 0.544282  , 0.7309462 ],
       [0.7240762 , 0.27932334, 0.99110806, 0.09674796, 0.8390946 ]],
      dtype=float32)

In [47]:
res_np==mult

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

In [48]:
res_np.shape

(8, 5)

In [49]:
mult.shape

(8, 5)

## MatMul (dense * sparse)

In [50]:
bdata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bdata)
bcols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcols)
bnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnz)
bdatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bdatat)
bcolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcolst)
bnnzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnzt)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // DENSE x SPARSE
    __kernel void matmul(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint   mwidth,
                            uint   ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);

      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        uint nnz = rowNnz[gid2];
        float sum = 0;
        for (uint i = 0; i < nnz; i++) {
          uint index   = (gid2 * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          float xval  = vector_x[gid*mwidth+col];
          sum  += aval * xval;
          if (gid==0 && gid2==0)
            printf("aval, xval: %.2f,%.2f - %.2f: (%i,%i) \\n", aval, xval, sum, col, index);
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid*ncols+gid2] = sum;
      }
    }""").build()

In [51]:
a.shape, b.shape

((8, 16), (16, 5))

In [52]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [53]:
rows = a.shape[0]

In [54]:
a.shape, b.shape

((8, 16), (16, 5))

In [55]:
mult = a.dot(b)
mult = mult.astype(np.float32)

In [56]:
outshape = np.array([a.shape[0], b.shape[1]])
outshape

array([8, 5])

In [57]:
b.T

array([[0.49395892, 0.65864426, 0.31044865, 0.        , 0.71082693,
        0.        , 0.46450275, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.18725161,
        0.7250254 ],
       [0.        , 0.        , 0.        , 0.46765924, 0.        ,
        0.        , 0.        , 0.34248227, 0.        , 0.7705159 ,
        0.7167741 , 0.91970533, 0.        , 0.        , 0.11607183,
        0.        ],
       [0.46451807, 0.9786625 , 0.02502257, 0.        , 0.58997136,
        0.84622455, 0.        , 0.7672639 , 0.9855889 , 0.31522992,
        0.        , 0.        , 0.        , 0.674503  , 0.        ,
        0.12346577],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.5916316 , 0.        , 0.        , 0.        , 0.        ,
        0.8238369 , 0.7873889 , 0.404568  , 0.48820347, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.03160795, 0.        ,
        0.        , 0.9429863 , 

In [58]:
a.T

array([[0.        , 0.9562663 , 0.        , 0.        , 0.        ,
        0.11415514, 0.28872353, 0.67626965],
       [0.68347037, 0.5531271 , 0.20103766, 0.        , 0.        ,
        0.82574075, 0.04889954, 0.        ],
       [0.8035886 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.14296253, 0.59893256,
        0.        , 0.6493426 , 0.35563806],
       [0.        , 0.5375557 , 0.        , 0.9103574 , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.74341315, 0.03242496, 0.        ,
        0.27978534, 0.33329687, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.69630474],
       [0.08023349, 0.        , 0.        , 0.71459514, 0.9217044 ,
        0.25018668, 0.        , 0.        ],
       [0.8858316 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.      

In [59]:
outshape.T

array([8, 5])

In [60]:
b.shape, outshape

((16, 5), array([8, 5]))

In [61]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod(outshape)*4)
knl = prg.matmul  # Use this Kernel object for repeated calls
knl(queue, [outshape.T[0]], None, bdatat_buf, bcolst_buf, bnnzst_buf, np.uint32(ellwbt), np.uint32(b.shape[0]), np.uint32(outshape.T[1]), a_buf, res_buf)

res_np = np.zeros(outshape).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)

aval, xval: 0.49,0.00 - 0.00: (0,0) 
aval, xval: 0.66,0.68 - 0.45: (1,1) 
aval, xval: 0.31,0.80 - 0.70: (2,2) 
aval, xval: 0.71,0.00 - 0.70: (4,3) 
aval, xval: 0.46,0.00 - 0.70: (6,4) 
aval, xval: 0.19,0.00 - 0.70: (14,5) 
aval, xval: 0.73,0.00 - 0.70: (15,6) 


In [62]:
(res_np-mult).sum()

0.0

In [63]:
a

array([[0.        , 0.68347037, 0.8035886 , 0.        , 0.        ,
        0.        , 0.        , 0.08023349, 0.8858316 , 0.        ,
        0.17861794, 0.        , 0.46201044, 0.        , 0.        ,
        0.        ],
       [0.9562663 , 0.5531271 , 0.        , 0.        , 0.5375557 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.20370705, 0.35530138, 0.        , 0.        ,
        0.9186601 ],
       [0.        , 0.20103766, 0.        , 0.        , 0.        ,
        0.74341315, 0.        , 0.        , 0.        , 0.5787496 ,
        0.        , 0.05387774, 0.        , 0.5917517 , 0.17797045,
        0.        ],
       [0.        , 0.        , 0.        , 0.14296253, 0.9103574 ,
        0.03242496, 0.        , 0.71459514, 0.        , 0.        ,
        0.        , 0.        , 0.7448136 , 0.        , 0.        ,
        0.66635793],
       [0.        , 0.        , 0.        , 0.59893256, 0.        ,
        0.        , 0.        , 

In [64]:
b

array([[0.49395892, 0.        , 0.46451807, 0.        , 0.        ],
       [0.65864426, 0.        , 0.9786625 , 0.        , 0.        ],
       [0.31044865, 0.        , 0.02502257, 0.        , 0.        ],
       [0.        , 0.46765924, 0.        , 0.        , 0.03160795],
       [0.71082693, 0.        , 0.58997136, 0.        , 0.        ],
       [0.        , 0.        , 0.84622455, 0.5916316 , 0.        ],
       [0.46450275, 0.        , 0.        , 0.        , 0.9429863 ],
       [0.        , 0.34248227, 0.7672639 , 0.        , 0.        ],
       [0.        , 0.        , 0.9855889 , 0.        , 0.2535647 ],
       [0.        , 0.7705159 , 0.31522992, 0.        , 0.        ],
       [0.        , 0.7167741 , 0.        , 0.8238369 , 0.        ],
       [0.        , 0.91970533, 0.        , 0.7873889 , 0.        ],
       [0.        , 0.        , 0.        , 0.404568  , 0.841323  ],
       [0.        , 0.        , 0.674503  , 0.48820347, 0.        ],
       [0.18725161, 0.11607183, 0.

In [65]:
res_buf

In [66]:
res_np

array([[0.6996368 , 0.15550727, 1.6236207 , 0.3340667 , 0.61331564],
       [1.8848312 , 0.18735047, 1.4160932 , 0.30414024, 0.29892322],
       [0.16573755, 0.51614475, 1.40742   , 0.7711447 , 0.        ],
       [1.130233  , 0.31159392, 1.1950791 , 0.32051137, 0.6311476 ],
       [0.49532753, 0.9677906 , 0.8187757 , 0.1351769 , 0.01893103],
       [0.60025734, 0.7728439 , 1.2898692 , 0.9285497 , 0.        ],
       [0.17482497, 0.30367106, 0.47157714, 0.544282  , 0.7309462 ],
       [0.7240762 , 0.27932334, 0.99110806, 0.09674796, 0.8390946 ]],
      dtype=float32)

In [67]:
mult

array([[0.6996368 , 0.15550727, 1.6236207 , 0.3340667 , 0.61331564],
       [1.8848312 , 0.18735047, 1.4160932 , 0.30414024, 0.29892322],
       [0.16573755, 0.51614475, 1.40742   , 0.7711447 , 0.        ],
       [1.130233  , 0.31159392, 1.1950791 , 0.32051137, 0.6311476 ],
       [0.49532753, 0.9677906 , 0.8187757 , 0.1351769 , 0.01893103],
       [0.60025734, 0.7728439 , 1.2898692 , 0.9285497 , 0.        ],
       [0.17482497, 0.30367106, 0.47157714, 0.544282  , 0.7309462 ],
       [0.7240762 , 0.27932334, 0.99110806, 0.09674796, 0.8390946 ]],
      dtype=float32)

In [68]:
res_np==mult

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

In [69]:
res_np-mult

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)

In [70]:
res_np.shape

(8, 5)

In [71]:
mult.shape

(8, 5)

## MatMul2 (dense * sparse)

wdata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=wdata)
wcols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=wcols)
wnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=wnnz)
wdatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=wdatat)
wcolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=wcolst)
wnnzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=wnnzt)
x_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=x_init)

prg = cl.Program(ctx, """
    // DENSE x SPARSE-T
    __kernel void matmul2(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint   mwidth,
                            uint   ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);
      uint nnz = rowNnz[gid];

      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        float sum = 0;
        for (uint i = 0; i < nnz; i++) {
          uint index   = (gid * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          float xval  = vector_x[gid2*ncols+col];
          sum  += aval * xval;
          if (gid==0 && gid2==1)
            printf("aval, xval: %.2f,%.2f - %.2f: (%i,%i) \\n", aval, xval, sum, col, index);
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid2*ncols+gid] = sum;
      }
    }""").build()

In [72]:
wdata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=wdata)
wcols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=wcols)
wnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=wnnz)
wdatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=wdatat)
wcolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=wcolst)
wnnzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=wnnzt)
x_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=x_init)

prg = cl.Program(ctx, """
    // DENSE x SPARSE-T
    __kernel void matmul2(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint   mwidth,
                            uint   ncols0,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);
      uint nnz = rowNnz[gid];
      uint gid2 = get_global_id(1);
      uint ncols = get_global_size(1);

      float sum = 0;
      for (uint i = 0; i < nnz; i++) {
        uint index   = (gid2 * ellwidth) + i;
        uint col     = colIdx[index];
        float aval  = matData[index];
        float xval  = vector_x[gid*mwidth+col];
        sum  += aval * xval;
        if (gid==1 && gid2==0) {
          printf("aval, xval: %.2f,%.2f - %.2f: (%i,%i) \\n", aval, xval, sum, col, index);
        }
      }
      //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
      vector_y[gid*ncols+gid2] = sum;
    }""").build()

In [73]:
outshape

array([8, 5])

In [74]:
w_init.shape, x_init.shape
w = w_init
x = x_init

In [75]:
res = np.zeros(w.shape[0]).astype(np.float32)
#res

In [76]:
rows = w.shape[0]

In [77]:
mult = mult.astype(np.float32)

In [78]:
outshape = np.array([x.shape[0], w.shape[0]])
outshape

array([ 8, 16])

In [79]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod(outshape)*4)
knl = prg.matmul2  # Use this Kernel object for repeated calls
knl(queue, outshape, None, wdata_buf, wcols_buf, wnnzs_buf, np.uint32(ellww), np.uint32(w.shape[1]), np.uint32(x.shape[1]), x_buf, res_buf)

res_np = np.zeros(outshape).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)

aval, xval: -0.66,-0.48 - 0.32: (0,0) 
aval, xval: -0.06,-1.52 - 0.40: (1,1) 
aval, xval: -0.50,-0.49 - 0.65: (2,2) 
aval, xval: 0.44,-0.24 - 0.54: (3,3) 
aval, xval: -0.38,-0.65 - 0.79: (4,4) 


In [80]:
mult = x.dot(w_init.T)
mult.shape

(8, 16)

In [81]:
mult

array([[ 0.71005666, -0.6922365 ,  1.958061  ,  1.258677  ,  3.463459  ,
        -0.3478019 ,  0.08231293, -0.04464556, -0.03240513,  1.8573855 ,
        -1.0835508 , -0.9068153 , -0.20014307, -1.5690712 , -1.1520894 ,
         0.6737664 ],
       [ 0.7850611 , -2.7718844 ,  3.8869042 ,  1.7455592 ,  2.9286547 ,
        -1.3900536 ,  0.8452302 ,  0.59935635,  0.31789836, -0.3896514 ,
        -0.80347735, -2.0294168 , -0.19058926, -1.6315975 , -3.2396262 ,
        -0.37664318],
       [-1.0399084 ,  3.7561824 , -8.091907  , -2.4692414 , -2.1408994 ,
         4.37048   , -1.9433106 ,  0.41747692,  0.1287814 ,  0.7065758 ,
         0.89383984,  3.3977637 , -0.99121714,  2.0500147 ,  5.0695367 ,
         0.4186813 ],
       [-0.46678752,  3.8533442 , -3.8412805 , -1.7131798 ,  1.1737914 ,
         3.4516523 , -0.6000567 , -0.39082292, -0.56545794,  1.7348484 ,
        -0.43604594,  3.096307  , -1.2181851 ,  0.24772911,  2.8284123 ,
         3.1381838 ],
       [-0.93206084, -0.5172032 , -3

In [82]:
res_np

array([[ 0.71005666, -0.6922365 ,  1.958061  ,  1.258677  ,  3.463459  ,
        -0.3478019 ,  0.08231293, -0.04464556, -0.03240513,  1.8573855 ,
        -1.0835508 , -0.9068153 , -0.20014307, -1.5690712 , -1.1520894 ,
         0.6737664 ],
       [ 0.7850611 , -2.7718844 ,  3.8869042 ,  1.7455592 ,  2.9286547 ,
        -1.3900536 ,  0.8452302 ,  0.59935635,  0.31789836, -0.3896514 ,
        -0.80347735, -2.0294168 , -0.19058926, -1.6315975 , -3.2396262 ,
        -0.37664318],
       [-1.0399084 ,  3.7561824 , -8.091907  , -2.4692414 , -2.1408994 ,
         4.37048   , -1.9433106 ,  0.41747692,  0.1287814 ,  0.7065758 ,
         0.89383984,  3.3977637 , -0.99121714,  2.0500147 ,  5.0695367 ,
         0.4186813 ],
       [-0.46678752,  3.8533442 , -3.8412805 , -1.7131798 ,  1.1737914 ,
         3.4516523 , -0.6000567 , -0.39082292, -0.56545794,  1.7348484 ,
        -0.43604594,  3.096307  , -1.2181851 ,  0.24772911,  2.8284123 ,
         3.1381838 ],
       [-0.93206084, -0.5172032 , -3

In [83]:
x

array([[ 1.1085547e-03, -2.8954408e-01, -1.1160663e+00, -1.2882757e-02,
        -3.7836146e-01],
       [-4.8113537e-01, -1.5173311e+00, -4.9087200e-01, -2.4068058e-01,
        -6.4794743e-01],
       [ 6.3589108e-01,  1.7401173e+00,  2.9668221e-01,  7.0750368e-01,
         1.8228158e+00],
       [ 4.3076903e-01,  1.5427296e+00, -9.0072119e-01, -1.3712502e-01,
         1.2975791e+00],
       [ 6.7527115e-01,  3.1958118e-02,  9.1814590e-01,  3.8050947e-01,
         5.1636750e-01],
       [-3.5523945e-01,  2.0877700e-01,  3.2841107e-01, -4.9822477e-01,
        -2.0917768e+00],
       [-8.2587741e-02,  2.4551826e+00, -2.6721101e+00, -9.1327929e-01,
        -2.2731435e-01],
       [ 2.6931539e-01,  1.1304612e+00,  1.0423975e+00,  1.3038105e+00,
         1.3894007e+00]], dtype=float32)

In [84]:
w

array([[-6.56452596e-01, -5.62572964e-02, -4.99902606e-01,
         4.36419368e-01, -3.75813037e-01],
       [-9.23061609e-01,  1.91725028e+00, -1.50302842e-01,
        -6.38729751e-01,  8.24770331e-01],
       [-1.21083879e+00, -5.03405392e-01, -7.01915681e-01,
        -1.97427106e+00, -2.65573215e+00],
       [-5.76822497e-02, -6.56186581e-01, -6.61706686e-01,
         7.69348443e-01, -8.99004877e-01],
       [ 1.69363797e+00, -1.69733524e+00, -2.79337025e+00,
        -2.26150647e-01,  3.97428840e-01],
       [ 1.65970361e+00, -4.93746817e-01, -3.76097679e-01,
        -1.69739768e-01,  2.41710639e+00],
       [-1.80884051e+00,  3.39751154e-01, -2.27297600e-02,
        -9.59997058e-01, -3.83114427e-01],
       [ 1.09529994e-01, -8.55162859e-01,  2.21606664e-04,
         6.63855076e-01,  7.49480963e-01],
       [-4.65818375e-01, -2.77439266e-01,  3.54995355e-02,
         8.48221183e-01,  1.62998557e-01],
       [ 1.20862365e+00,  5.02520800e-01, -1.58382213e+00,
         1.02303350e+00

In [85]:
(res_np-mult).sum()

0.0

In [86]:
mult

array([[ 0.71005666, -0.6922365 ,  1.958061  ,  1.258677  ,  3.463459  ,
        -0.3478019 ,  0.08231293, -0.04464556, -0.03240513,  1.8573855 ,
        -1.0835508 , -0.9068153 , -0.20014307, -1.5690712 , -1.1520894 ,
         0.6737664 ],
       [ 0.7850611 , -2.7718844 ,  3.8869042 ,  1.7455592 ,  2.9286547 ,
        -1.3900536 ,  0.8452302 ,  0.59935635,  0.31789836, -0.3896514 ,
        -0.80347735, -2.0294168 , -0.19058926, -1.6315975 , -3.2396262 ,
        -0.37664318],
       [-1.0399084 ,  3.7561824 , -8.091907  , -2.4692414 , -2.1408994 ,
         4.37048   , -1.9433106 ,  0.41747692,  0.1287814 ,  0.7065758 ,
         0.89383984,  3.3977637 , -0.99121714,  2.0500147 ,  5.0695367 ,
         0.4186813 ],
       [-0.46678752,  3.8533442 , -3.8412805 , -1.7131798 ,  1.1737914 ,
         3.4516523 , -0.6000567 , -0.39082292, -0.56545794,  1.7348484 ,
        -0.43604594,  3.096307  , -1.2181851 ,  0.24772911,  2.8284123 ,
         3.1381838 ],
       [-0.93206084, -0.5172032 , -3

In [87]:
res_np==mult

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  Tru

In [88]:
res_np-mult

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [89]:
res_np.shape

(8, 16)

In [90]:
mult.shape

(8, 16)

In [91]:
asdf

NameError: name 'asdf' is not defined

## MatMul (dense * sparse) NEW

bdata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bdata)
bcols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcols)
bnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnz)
bdatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bdatat)
bcolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcolst)
bnnzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnzt)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // DENSE x SPARSE
    __kernel void matmulnew(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint   mwidth,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);
      uint gid2 = get_global_id(1);
      uint ncols = get_global_size(1);
      uint nnz = rowNnz[gid2];
      float sum = 0;
      for (uint i = 0; i < nnz; i++) {
        uint index   = (gid2 * ellwidth) + i;
        uint col     = colIdx[index];
        float aval  = matData[index];
        float xval  = vector_x[gid*mwidth+col];
        vector_y[gid2*nrows+gid] += aval * xval;
        if (gid==0 && gid2==0)
          printf("aval, xval: %.2f,%.2f - %.2f: (%i,%i) \\n", aval, xval, sum, col, index);
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        
      }
      
    }""").build()

In [ ]:
mult = a.dot(b)
mult

In [ ]:
bdata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=data)
bcols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcols)
bnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnz)
bdatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bdatat)
bcolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcolst)
bnnzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnzt)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // DENSE x SPARSE
    __kernel void matmulnew(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint   mwidth,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);
      uint gid2 = get_global_id(1);
      uint ncols = get_global_size(1);
      uint nnz = rowNnz[gid2];
      float sum = 0;
      for (uint i = 0; i < nnz; i++) {
        uint index   = (gid2 * ellwidth) + i;
        uint col     = colIdx[index];
        float aval  = matData[index];
        float xval  = vector_x[gid*mwidth+col];
        sum  += aval * xval;
        if (gid==1 && gid2==0)
          printf("aval, xval: %.2f,%.2f - %.2f: (%i,%i) \\n", aval, xval, sum, col, index);
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
      }
      vector_y[gid2*ncols+gid] = sum;
    }""").build()

In [ ]:
a.shape, b.shape

In [ ]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [ ]:
rows = a.shape[0]

In [ ]:
mult = mult.astype(np.float32)

In [ ]:
outshape = np.array([a.shape[0], b.shape[1]])
outshape

In [ ]:
b.T

In [ ]:
a

In [ ]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod(outshape)*4)
knl = prg.matmulnew  # Use this Kernel object for repeated calls
knl(queue, outshape, None, bdatat_buf, bcolst_buf, bnnzst_buf, np.uint32(ellwbt), np.uint32(b.shape[0]), a_buf, res_buf)

res_np = np.zeros((outshape[0],)).astype(np.float32)
print(res_np.shape)
cl.enqueue_copy(queue, res_np, res_buf)

In [ ]:
(res_np-mult.T).sum()

In [ ]:
res_buf

In [ ]:
res_np.T

In [ ]:
mult

In [ ]:
res_np-mult.T

In [ ]:
res_np.shape

In [ ]:
mult.shape

In [ ]:
asdf

# Matmult Dense Dense

In [ ]:
b_buf2 = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // multilplies x by y WITH Y TRANSPOSED INDEXING
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      uint osize = get_global_size(1);
      int gidx = get_global_id(0); // row
      int gidy = get_global_id(1); // col

      float ret = 0.0;
      for (int i = 0; i < msize; i++) {
        uint xidx = gidx*msize+i; 
        float xval = x[xidx];
        uint yidx = osize*i+gidy;
        float yval = y[yidx];
        ret += xval*yval;
        if (gidx==0 && gidy==0)
          printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, res, xidx, yidx);
      }

      //if (gidx==0&&gidy==0)
      //  printf("\\nsum:%.2f", ret);
      res[gidx * osize + gidy] = ret;
    }""").build()

In [ ]:
a.shape, b.shape

In [ ]:
rows = a.shape[0]

In [ ]:
mult = mult.astype(np.float32)

In [ ]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [rows,b.shape[1]], None, a_buf, b_buf2, res_buf, np.uint32(a.shape[1]))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)

In [ ]:
(res_np-mult.T).sum()

In [ ]:
a

In [ ]:
b

In [ ]:
res_np

In [ ]:
a.dot(b)

In [ ]:
res_np==mult

In [ ]:
res_np.shape

In [ ]:
mult.shape

# Matmult Dense Transposed

In [ ]:
b

In [ ]:
c=np.zeros(b.T.shape)
bt = b.T
for row in range(bt.shape[0]):
    for col in range(bt.shape[1]):
        c[row][col] = bt[row][col]

In [ ]:
b_buf2 = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // multilplies x by y WITH Y TRANSPOSED INDEXING
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      uint osize = get_global_size(1);
      int gidx = get_global_id(0); // row
      int gidy = get_global_id(1); // col

      float ret = 0.0;
      for (int i = 0; i < msize; i++) {
        uint xidx = gidx*msize+i;
        float xval = x[xidx];
        uint yidx = msize*gidy+i;
        float yval = y[yidx];
        ret += xval*yval;
        if (gidx==0 && gidy==0)
          printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, res, xidx, yidx);
      }

      //if (gidx==0&&gidy==0)
      //  printf("\\nsum:%.2f", ret);
      res[gidx * osize + gidy] = ret;
    }""").build()

In [ ]:
a.shape, b.T.shape

In [ ]:
rows = a.shape[0]

In [ ]:
mult = mult.astype(np.float32)

In [ ]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [rows,b.shape[1]], None, a_buf, b_buf2, res_buf, np.uint32(a.shape[1]))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)

In [ ]:
(res_np-mult).sum()

In [ ]:
a

In [ ]:
b

In [ ]:
res_np

In [ ]:
a.dot(b)

In [ ]:
res_np==mult

In [ ]:
res_np.shape

In [ ]:
mult.shape

# Matmult Transposed Dense

In [ ]:
mult = a.dot(b)

In [ ]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [ ]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // multilplies x TRANSPOSED by y (dense-dense)
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize,
                          uint isize
                          ) { // LOCAL SHARED BUFFER
      uint osize = get_global_size(0);
      int gidy = get_global_id(0); // row
      
      for (uint gidx = 0; gidx < isize; gidx++) {
        float ret = 0.0;
        for (uint i = 0; i < msize; i++) {
          uint xidx = i*isize+gidx;
          float xval = x[xidx];
          uint yidx = osize*i+gidy;
          float yval = y[yidx];
          ret += xval*yval;
          if (gidx==0 && gidy==0)
            printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, ret, xidx, yidx);
        }
        //if (gidx==0&&gidy==0)
        //  printf("\\nsum:%.2f", ret);
        res[gidx * osize + gidy] = ret;
      }
    }""").build()

In [ ]:
a.shape, b.shape

In [ ]:
rows = a.shape[0]

In [ ]:
mult = mult.astype(np.float32)
mult.shape

In [ ]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [b.shape[1]], None, a_buf, b_buf, res_buf, np.uint32(a.shape[1]), np.uint32(rows))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)

In [ ]:
(res_np-mult).sum()

In [ ]:
a

In [ ]:
b

In [ ]:
res_np

In [ ]:
a.dot(b)

In [ ]:
res_np==mult

In [ ]:
res_np.shape

In [ ]:
mult.shape

# Matmult Transposed Dense (SPR) - Get Topk - NEW

In [793]:
topkx = 3
topky = 3

In [794]:
mult.shape

(8, 5)

In [831]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
x_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, a.shape[0]*4)
y_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.shape[1]*4)
xs_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topkx*4)
ys_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topky*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topkx*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topky*4)

prg = cl.Program(ctx, """
    // multilplies x TRANSPOSED by y (dense-dense)
    __kernel void gettopk(__global  float* x,      // INPUT MATRIX DATA
                          __global  float* y,      // INPUT
                          __global  float* xsum,    // INPUT
                          __global  float* ysum,    // INPUT
                          __global  uint*  xoutidx, // OUT
                          __global  uint*  youtidx, // OUT
                          __global  uint*  xsoutidx, // OUT
                          __global  uint*  ysoutidx, // OUT
                          uint topkx,
                          uint topky,
                          uint msize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      int gidx = get_global_id(0); // row
      uint osize = get_global_size(1);
      int gidy = get_global_id(1); // row

      
      // get topk
      if (gidy==0) {
        xsum[gidx] = 0;
        for (uint i=0; i<msize; i++) {
          float val = x[i*isize+gidx];
          //if (gid == 0) {
          //  printf("\\nADD VALx: %.2f - %i", val, i*msize+gid);
          //}
          xsum[gidx] += val;
        }
        float valx = xsum[gidx];
        uint posx = 0;
        for (uint i = 0; i < isize; i++) {
          float tempval = fabs(xsum[i]);
          bool larger = tempval > fabs(valx);
          posx += (larger)?1:0;
        }
        if (posx < topky) {
          youtidx[posx] = gidx;
        }
      }
      
      if (gidx==0) {
        ysum[gidy] = 0;
        for (uint i=0; i<msize; i++) {
          float val = y[i*osize+gidy];
          //if (gid == 0) {
          //  printf("\\nADD VALx: %.2f - %i", val, i*msize+gid);
          //}
          ysum[gidy] += val;
        }
        float valy = ysum[gidy];
        uint posy = 0;
        for (uint i = 0; i < osize; i++) {
          float tempval = fabs(ysum[i]);
          bool larger = tempval > fabs(valy);
          posy += (larger)?1:0;
        }
        if (posy < topkx) {
          xoutidx[posy] = gidy;
        }
      }
      
      
      // sort results
      if (gidx < topky && gidy==0) {
        uint valx2 = youtidx[gidx];
        uint posx = 0;
        for (uint i = 0; i < topky; i++) {
          uint tempval = youtidx[i];
          bool larger = tempval < valx2;
          posx += (larger)?1:0;
        }
        ysoutidx[posx] = youtidx[gidx];
      }
      
      if (gidy < topkx && gidx==0) {
        uint valy2 = xoutidx[gidy];
        printf("VAL2:%.2f",valy2);
        uint posy = 0;
        for (uint i = 0; i < topkx; i++) {
          uint tempval = xoutidx[i];
          bool larger = tempval < valy2;
          posy += (larger)?1:0;
        }
        xsoutidx[posy] = xoutidx[gidy];
      }
    }""").build()

In [832]:
topkx, topky

(3, 3)

In [833]:
rows = a.shape[0]
cols = b.shape[1]
rows, cols

(8, 5)

In [834]:
mult = mult.astype(np.float32)
mult.shape

(8, 5)

In [835]:
resdata_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
rescols_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
resnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows])*4)

knl = prg.gettopk  # Use this Kernel object for repeated calls
knl(queue, [rows,cols], None, a_buf, b_buf, x_sum_buf, y_sum_buf, x_idx_buf, y_idx_buf, xs_idx_buf, ys_idx_buf, np.uint32(topkx), np.uint32(topky), np.uint32(a.shape[1]))

xsum = np.zeros(a.shape[0]).astype(np.float32)
ysum = np.zeros(b.shape[1]).astype(np.float32)
xidxcols = np.zeros(topkx).astype(np.uint32)
yidxcols = np.zeros(topky).astype(np.uint32)

cl.enqueue_copy(queue, xsum, x_sum_buf)
cl.enqueue_copy(queue, ysum, y_sum_buf)
cl.enqueue_copy(queue, xidxcols, xs_idx_buf)
cl.enqueue_copy(queue, yidxcols, ys_idx_buf)

VAL2:0.00VAL2:0.00VAL2:0.00

In [836]:
yidxcols

array([1, 3, 4], dtype=uint32)

In [837]:
a.sum(axis=1)

array([3.0937524, 3.5246177, 2.3468003, 3.2115116, 3.102983 , 2.4004393,
       2.1758807, 2.6182923], dtype=float32)

In [838]:
xsum

array([3.0937524, 3.5246177, 2.3468003, 3.2115116, 3.102983 , 2.4004393,
       2.1758807, 2.6182923], dtype=float32)

In [839]:
ysum

array([3.5506585, 3.3332088, 5.77045  , 3.095629 , 2.0694818],
      dtype=float32)

In [840]:
xidxcols

array([0, 1, 2], dtype=uint32)

In [841]:
yidxcols

array([1, 3, 4], dtype=uint32)

# Matmult Transposed Dense (SPRNEW)

In [842]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [971]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // multilplies x TRANSPOSED by y (dense-dense)
    __kernel void matmul0(__global  float* x,      // INPUT MATRIX DATA
                          __global  float* y,      // INPUT
                          __global  uint* xidx,   // INPUT YIDX
                          __global  uint* yidx,   // INPUT YIDX
                          __global  float* resdata,// OUT
                          __global  uint*  rescols,
                          __global  uint*  resnnzs,
                          uint topkx,
                          uint ellw,
                          uint isize,
                          uint msize,
                          uint osize
                          ) { // LOCAL SHARED BUFFER  
                          
      uint topky = get_global_size(0);
      uint gidx = yidx[get_global_id(0)]; // row
      
      for (uint gidy0 = 0; gidy0 < topkx; gidy0++) {
        uint gidy = xidx[gidy0];
        float ret = 0.0;
        uint i;
        for (i = 0; i < msize; i++) {
          uint xidx = i*isize+gidx;
          float xval = x[xidx];
          uint yidx = osize*i+gidy;
          float yval = y[yidx];
          ret += xval*yval;
          //if (gidx==0 && gidy==0)
          //  printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, ret, xidx, yidx);
        }
        //if (gidx==0&&gidy==0)
        //  printf("\\nsum:%.2f", ret);
        
        // add for 
        uint nnz = resnnzs[gidx];
        for (i = 0; i < nnz; i++) {
          if (rescols[i] >= gidy) {
            break;
          }
          for (uint j = nnz; j >= i; j--) {
            //resdata[j+1] = resdata[j];
          }
        }
        resdata[gidx * ellw + gidy0] = ret;
        rescols[gidx * ellw + gidy0] = gidy;
        resnnzs[gidx] += 1;
      }
    }""").build()

In [972]:
a.shape, b.shape

((8, 16), (16, 5))

In [973]:
rows = a.shape[0]

In [974]:
mult = mult.astype(np.float32)
mult.shape

(8, 5)

In [975]:
topky, topkx

(3, 3)

In [976]:
resdata_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
rescols_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
resnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=np.zeros(rows))

knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [topky], None, a_buf, b_buf, xs_idx_buf, ys_idx_buf, resdata_buf, rescols_buf, resnnzs_buf, np.uint32(topkx), np.uint32(b.shape[1]), np.uint32(a.shape[0]), np.uint32(a.shape[1]), np.uint32(b.shape[1]))

resdata = np.zeros(a.shape[0]*b.shape[1]).astype(np.float32)
rescols = np.zeros(a.shape[0]*b.shape[1]).astype(np.uint32)
resnnzs = np.zeros(a.shape[0]).astype(np.uint32)
cl.enqueue_copy(queue, resdata, resdata_buf)
cl.enqueue_copy(queue, rescols, rescols_buf)
cl.enqueue_copy(queue, resnnzs, resnnzs_buf)

In [977]:
resdata

array([0.6996368 , 1.6236207 , 1.6236207 , 0.3340667 , 0.61331564,
       1.8848312 , 0.18735047, 1.4160932 , 0.30414024, 0.29892322,
       0.16573755, 0.51614475, 1.40742   , 0.7711447 , 0.        ,
       1.130233  , 0.31159392, 1.1950791 , 0.32051137, 0.6311476 ,
       0.49532753, 0.9677906 , 0.8187757 , 0.1351769 , 0.01893103,
       0.60025734, 0.7728439 , 1.2898692 , 0.9285497 , 0.        ,
       0.17482497, 0.30367106, 0.47157714, 0.544282  , 0.7309462 ,
       0.7240762 , 0.27932334, 0.99110806, 0.09674796, 0.8390946 ],
      dtype=float32)

In [978]:
rescols

array([0, 2, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1,
       2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4], dtype=uint32)

In [979]:
resnnzs

array([0, 3, 0, 3, 3, 0, 0, 0], dtype=uint32)

In [980]:
res_np = to_dense(resdata, rescols, resnnzs, b.shape[1], mult.shape)

In [981]:
(res_np-mult).sum()

-16.742596236988902

In [982]:
a

array([[0.        , 0.68347037, 0.8035886 , 0.        , 0.        ,
        0.        , 0.        , 0.08023349, 0.8858316 , 0.        ,
        0.17861794, 0.        , 0.46201044, 0.        , 0.        ,
        0.        ],
       [0.9562663 , 0.5531271 , 0.        , 0.        , 0.5375557 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.20370705, 0.35530138, 0.        , 0.        ,
        0.9186601 ],
       [0.        , 0.20103766, 0.        , 0.        , 0.        ,
        0.74341315, 0.        , 0.        , 0.        , 0.5787496 ,
        0.        , 0.05387774, 0.        , 0.5917517 , 0.17797045,
        0.        ],
       [0.        , 0.        , 0.        , 0.14296253, 0.9103574 ,
        0.03242496, 0.        , 0.71459514, 0.        , 0.        ,
        0.        , 0.        , 0.7448136 , 0.        , 0.        ,
        0.66635793],
       [0.        , 0.        , 0.        , 0.59893256, 0.        ,
        0.        , 0.        , 

In [983]:
b

array([[0.49395892, 0.        , 0.46451807, 0.        , 0.        ],
       [0.65864426, 0.        , 0.9786625 , 0.        , 0.        ],
       [0.31044865, 0.        , 0.02502257, 0.        , 0.        ],
       [0.        , 0.46765924, 0.        , 0.        , 0.03160795],
       [0.71082693, 0.        , 0.58997136, 0.        , 0.        ],
       [0.        , 0.        , 0.84622455, 0.5916316 , 0.        ],
       [0.46450275, 0.        , 0.        , 0.        , 0.9429863 ],
       [0.        , 0.34248227, 0.7672639 , 0.        , 0.        ],
       [0.        , 0.        , 0.9855889 , 0.        , 0.2535647 ],
       [0.        , 0.7705159 , 0.31522992, 0.        , 0.        ],
       [0.        , 0.7167741 , 0.        , 0.8238369 , 0.        ],
       [0.        , 0.91970533, 0.        , 0.7873889 , 0.        ],
       [0.        , 0.        , 0.        , 0.404568  , 0.841323  ],
       [0.        , 0.        , 0.674503  , 0.48820347, 0.        ],
       [0.18725161, 0.11607183, 0.

In [984]:
res_np

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [1.88483119, 0.18735047, 1.41609323, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [1.13023305, 0.31159392, 1.19507909, 0.        , 0.        ],
       [0.49532753, 0.9677906 , 0.81877571, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [985]:
a.dot(b)

array([[0.6996368 , 0.15550727, 1.6236207 , 0.3340667 , 0.61331564],
       [1.8848312 , 0.18735047, 1.4160932 , 0.30414024, 0.29892322],
       [0.16573755, 0.51614475, 1.40742   , 0.7711447 , 0.        ],
       [1.130233  , 0.31159392, 1.1950791 , 0.32051137, 0.6311476 ],
       [0.49532753, 0.9677906 , 0.8187757 , 0.1351769 , 0.01893103],
       [0.60025734, 0.7728439 , 1.2898692 , 0.9285497 , 0.        ],
       [0.17482497, 0.30367106, 0.47157714, 0.544282  , 0.7309462 ],
       [0.7240762 , 0.27932334, 0.99110806, 0.09674796, 0.8390946 ]],
      dtype=float32)

In [986]:
res_np==mult

array([[False, False, False, False, False],
       [ True,  True,  True, False, False],
       [False, False, False, False,  True],
       [ True,  True,  True, False, False],
       [ True,  True,  True, False, False],
       [False, False, False, False,  True],
       [False, False, False, False, False],
       [False, False, False, False, False]])

In [987]:
res_np.shape

(8, 5)

In [988]:
mult.shape

(8, 5)

# Matmult Transposed Dense (SPR-T OUT NEW)

In [989]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [954]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // multilplies x TRANSPOSED by y (dense-dense)
    __kernel void matmul0t(__global  float* x,      // INPUT MATRIX DATA
                          __global  float* y,      // INPUT
                          __global  uint* xidx,   // INPUT YIDX
                          __global  uint* yidx,   // INPUT YIDX
                          __global  float* resdata,// OUT
                          __global  uint*  rescols,
                          __global  uint*  resnnzs,
                          uint topky,
                          uint ellw,
                          uint isize,
                          uint msize,
                          uint osize
                          ) { // LOCAL SHARED BUFFER
      uint topkx = get_global_size(0);
      uint gidy = xidx[get_global_id(0)]; // row
      
      for (uint gidx0 = 0; gidy0 < topky; gidy0++) {
        uint gidx = yidx[gidy0];
        float ret = 0.0;
        uint i;
        for (i = 0; i < msize; i++) {
          uint xidx = i*isize+gidx;
          float xval = x[xidx];
          uint yidx = osize*i+gidy;
          float yval = y[yidx];
          ret += xval*yval;
          if (gidx==0 && gidy==0)
            printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, ret, gidx, gidy,i);
        }
        //if (gidx==0&&gidy==0)
        //  printf("\\nsum:%.2f", ret);
        
        // add for 
        uint nnz = resnnzs[gidx];
        for (i = 0; i < nnz; i++) {
          if (rescols[i] >= gidy) {
            break;
          }
          for (uint j = nnz; j >= i; j--) {
            //resdata[j+1] = resdata[j];
          }
        }
        resdata[gidy * ellw + gidx] = ret;
        rescols[gidy * ellw + gidx] = gidx;
        resnnzs[gidy] += 1;
      }
    }""").build()

In [955]:
a.shape, b.shape

((8, 16), (16, 5))

In [956]:
rows = a.shape[0]
cols = b.shape[1]

In [957]:
mult = mult.astype(np.float32)
mult.shape

(8, 5)

In [958]:
resdatat_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([cols,rows])*4)
rescolst_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([cols,rows])*4)
resnnzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=np.zeros(cols))

knl = prg.matmul0t  # Use this Kernel object for repeated calls
knl(queue, [topkx], None, a_buf, b_buf, xs_idx_buf, ys_idx_buf, resdatat_buf, rescolst_buf, resnnzst_buf, np.uint32(topky), np.uint32(a.shape[0]), np.uint32(a.shape[0]), np.uint32(a.shape[1]), np.uint32(b.shape[1]))

resdatat = np.zeros(cols*rows).astype(np.float32)
rescolst = np.zeros(cols*rows).astype(np.uint32)
resnnzst = np.zeros(cols).astype(np.uint32)
cl.enqueue_copy(queue, resdatat, resdatat_buf)
cl.enqueue_copy(queue, rescolst, rescolst_buf)
cl.enqueue_copy(queue, resnnzst, resnnzst_buf)

In [959]:
a

array([[0.        , 0.68347037, 0.8035886 , 0.        , 0.        ,
        0.        , 0.        , 0.08023349, 0.8858316 , 0.        ,
        0.17861794, 0.        , 0.46201044, 0.        , 0.        ,
        0.        ],
       [0.9562663 , 0.5531271 , 0.        , 0.        , 0.5375557 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.20370705, 0.35530138, 0.        , 0.        ,
        0.9186601 ],
       [0.        , 0.20103766, 0.        , 0.        , 0.        ,
        0.74341315, 0.        , 0.        , 0.        , 0.5787496 ,
        0.        , 0.05387774, 0.        , 0.5917517 , 0.17797045,
        0.        ],
       [0.        , 0.        , 0.        , 0.14296253, 0.9103574 ,
        0.03242496, 0.        , 0.71459514, 0.        , 0.        ,
        0.        , 0.        , 0.7448136 , 0.        , 0.        ,
        0.66635793],
       [0.        , 0.        , 0.        , 0.59893256, 0.        ,
        0.        , 0.        , 

In [960]:
b.T

array([[0.49395892, 0.65864426, 0.31044865, 0.        , 0.71082693,
        0.        , 0.46450275, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.18725161,
        0.7250254 ],
       [0.        , 0.        , 0.        , 0.46765924, 0.        ,
        0.        , 0.        , 0.34248227, 0.        , 0.7705159 ,
        0.7167741 , 0.91970533, 0.        , 0.        , 0.11607183,
        0.        ],
       [0.46451807, 0.9786625 , 0.02502257, 0.        , 0.58997136,
        0.84622455, 0.        , 0.7672639 , 0.9855889 , 0.31522992,
        0.        , 0.        , 0.        , 0.674503  , 0.        ,
        0.12346577],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.5916316 , 0.        , 0.        , 0.        , 0.        ,
        0.8238369 , 0.7873889 , 0.404568  , 0.48820347, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.03160795, 0.        ,
        0.        , 0.9429863 , 

In [961]:
resdatat

array([0.6996368 , 1.8848312 , 0.16573755, 1.130233  , 0.49532753,
       0.60025734, 0.17482497, 0.7240762 , 1.8848312 , 0.18735047,
       1.4160932 , 0.31159392, 0.9677906 , 0.7728439 , 0.30367106,
       0.27932334, 1.6236207 , 1.4160932 , 1.40742   , 1.1950791 ,
       0.8187757 , 1.2898692 , 0.47157714, 0.99110806, 1.130233  ,
       0.31159392, 1.1950791 , 0.32051137, 0.1351769 , 0.9285497 ,
       0.544282  , 0.09674796, 0.49532753, 0.9677906 , 0.8187757 ,
       0.6311476 , 0.01893103, 0.        , 0.7309462 , 0.8390946 ],
      dtype=float32)

In [962]:
rescolst

array([0, 1, 2, 3, 4, 5, 6, 7, 1, 1, 1, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5,
       6, 7, 3, 3, 3, 3, 4, 5, 6, 7, 4, 4, 4, 3, 4, 5, 6, 7], dtype=uint32)

In [963]:
resnnzst

array([3, 3, 3, 0, 0], dtype=uint32)

In [964]:
res_np = to_dense(resdatat, rescolst, resnnzst, a.shape[0], mult.T.shape)
res_np.T

array([[0.69963682, 0.        , 1.62362075, 0.        , 0.        ],
       [1.88483119, 1.41609323, 1.41609323, 0.        , 0.        ],
       [0.16573755, 0.        , 1.40742004, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [965]:
(res_np.T-mult).sum()

-16.53623822145164

In [966]:
res_np.shape

(5, 8)

In [967]:
a.dot(b)

array([[0.6996368 , 0.15550727, 1.6236207 , 0.3340667 , 0.61331564],
       [1.8848312 , 0.18735047, 1.4160932 , 0.30414024, 0.29892322],
       [0.16573755, 0.51614475, 1.40742   , 0.7711447 , 0.        ],
       [1.130233  , 0.31159392, 1.1950791 , 0.32051137, 0.6311476 ],
       [0.49532753, 0.9677906 , 0.8187757 , 0.1351769 , 0.01893103],
       [0.60025734, 0.7728439 , 1.2898692 , 0.9285497 , 0.        ],
       [0.17482497, 0.30367106, 0.47157714, 0.544282  , 0.7309462 ],
       [0.7240762 , 0.27932334, 0.99110806, 0.09674796, 0.8390946 ]],
      dtype=float32)

In [968]:
mult - res_np.T

array([[ 0.        ,  0.15550727,  0.        ,  0.33406669,  0.61331564],
       [ 0.        , -1.22874276,  0.        ,  0.30414024,  0.29892322],
       [ 0.        ,  0.51614475,  0.        ,  0.77114469,  0.        ],
       [ 1.13023305,  0.31159392,  1.19507909,  0.32051137,  0.63114762],
       [ 0.49532753,  0.9677906 ,  0.81877571,  0.1351769 ,  0.01893103],
       [ 0.60025734,  0.7728439 ,  1.28986919,  0.92854971,  0.        ],
       [ 0.17482497,  0.30367106,  0.47157714,  0.54428202,  0.73094618],
       [ 0.72407621,  0.27932334,  0.99110806,  0.09674796,  0.83909458]])

In [969]:
res_np.T==mult

array([[ True, False,  True, False, False],
       [ True, False,  True, False, False],
       [ True, False,  True, False,  True],
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False,  True],
       [False, False, False, False, False],
       [False, False, False, False, False]])

In [970]:
res_np.T

array([[0.69963682, 0.        , 1.62362075, 0.        , 0.        ],
       [1.88483119, 1.41609323, 1.41609323, 0.        , 0.        ],
       [0.16573755, 0.        , 1.40742004, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

# Matmult Transposed Dense (SPR)

In [ ]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [ ]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // multilplies x TRANSPOSED by y (dense-dense)
    __kernel void matmul0(__global  float* x,      // INPUT MATRIX DATA
                          __global  float* y,      // INPUT
                          __global  float* resdata,// OUT
                          __global  uint*  rescols,
                          __global  uint*  resnnzs,
                          uint ellw,
                          uint msize,
                          uint osize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      int gidx = get_global_id(0); // row
      
      resnnzs[gidx] = 0;
      
      for (uint gidy = 0; gidy < osize; gidy++) {
        float ret = 0.0;
        uint i;
        for (i = 0; i < msize; i++) {
          uint xidx = i*isize+gidx;
          float xval = x[xidx];
          uint yidx = osize*i+gidy;
          float yval = y[yidx];
          ret += xval*yval;
          if (gidx==0 && gidy==0)
            printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, ret, xidx, yidx);
        }
        //if (gidx==0&&gidy==0)
        //  printf("\\nsum:%.2f", ret);
        
        // add for 
        uint nnz = resnnzs[gidx];
        for (i = 0; i < nnz; i++) {
          if (rescols[i] >= gidy) {
            break;
          }
          for (uint j = nnz; j >= i; j--) {
            //resdata[j+1] = resdata[j];
          }
        }
        resdata[gidx * ellw + i] = ret;
        rescols[gidx * ellw + i] = gidy;
        resnnzs[gidx] += 1;
      }
    }""").build()

In [ ]:
a.shape, b.shape

In [ ]:
rows = a.shape[0]

In [ ]:
mult = mult.astype(np.float32)
mult.shape

In [ ]:
resdata_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
rescols_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
resnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows])*4)

knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [rows], None, a_buf, b_buf, resdata_buf, rescols_buf, resnnzs_buf, np.uint32(b.shape[1]), np.uint32(a.shape[1]), np.uint32(b.shape[1]))

resdata = np.zeros(a.shape[0]*b.shape[1]).astype(np.float32)
rescols = np.zeros(a.shape[0]*b.shape[1]).astype(np.uint32)
resnnzs = np.zeros(a.shape[0]).astype(np.uint32)
cl.enqueue_copy(queue, resdata, resdata_buf)
cl.enqueue_copy(queue, rescols, rescols_buf)
cl.enqueue_copy(queue, resnnzs, resnnzs_buf)

In [ ]:
resdata

In [ ]:
rescols

In [ ]:
resnnzs

In [ ]:
res_np = to_dense(resdata, rescols, resnnzs, b.shape[1], mult.shape)

In [ ]:
(res_np-mult).sum()

In [ ]:
a

In [ ]:
b

In [ ]:
res_np

In [ ]:
a.dot(b)

In [ ]:
res_np==mult

In [92]:
res_np.shape

(8, 16)

In [93]:
mult.shape

(8, 16)

# Matmult Transposed Dense (SPR-T OUT)

In [94]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [95]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // multilplies x TRANSPOSED by y (dense-dense)
    __kernel void matmul0t(__global  float* x,      // INPUT MATRIX DATA
                          __global  float* y,      // INPUT
                          __global  float* resdata,// OUT
                          __global  uint*  rescols,
                          __global  uint*  resnnzs,
                          uint ellw,
                          uint msize,
                          uint isize
                          ) { // LOCAL SHARED BUFFER
      uint osize = get_global_size(0);
      int gidy = get_global_id(0); // row
      
      resnnzs[gidy] = 0;
      for (uint gidx = 0; gidx < isize; gidx++) {
        float ret = 0.0;
        uint i;
        for (i = 0; i < msize; i++) {
          uint xidx = i*isize+gidx;
          float xval = x[xidx];
          uint yidx = osize*i+gidy;
          float yval = y[yidx];
          ret += xval*yval;
          if (gidx==0 && gidy==0)
            printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, ret, xidx, yidx);
        }
        //if (gidx==0&&gidy==0)
        //  printf("\\nsum:%.2f", ret);
        
        // add for 
        uint nnz = resnnzs[gidy];
        for (i = 0; i < nnz; i++) {
          if (rescols[i] >= gidx) {
            break;
          }
          for (uint j = nnz; j >= i; j--) {
            //resdata[j+1] = resdata[j];
          }
        }
        resdata[gidy * ellw + i] = ret;
        rescols[gidy * ellw + i] = gidx;
        resnnzs[gidy] += 1;
      }
    }""").build()

In [96]:
a.shape, b.shape

((8, 16), (16, 5))

In [97]:
rows = a.shape[0]
cols = b.shape[1]

In [98]:
mult = mult.astype(np.float32)
mult.shape

(8, 16)

In [99]:
resdatat_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([cols,rows])*4)
rescolst_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([cols,rows])*4)
resnnzst_buf = cl.Buffer(ctx, mf.READ_WRITE, cols*4)

knl = prg.matmul0t  # Use this Kernel object for repeated calls
knl(queue, [cols], None, a_buf, b_buf, resdatat_buf, rescolst_buf, resnnzst_buf, np.uint32(rows), np.uint32(a.shape[1]), np.uint32(rows))

resdatat = np.zeros(cols*rows).astype(np.float32)
rescolst = np.zeros(cols*rows).astype(np.uint32)
resnnzst = np.zeros(cols).astype(np.uint32)
cl.enqueue_copy(queue, resdatat, resdatat_buf)
cl.enqueue_copy(queue, rescolst, rescolst_buf)
cl.enqueue_copy(queue, resnnzst, resnnzst_buf)


mult: 0.00 x 0.49 - 0.00  -- 0/0
mult: 0.68 x 0.66 - 0.45  -- 8/5
mult: 0.80 x 0.31 - 0.70  -- 16/10
mult: 0.00 x 0.00 - 0.70  -- 24/15
mult: 0.00 x 0.71 - 0.70  -- 32/20
mult: 0.00 x 0.00 - 0.70  -- 40/25
mult: 0.00 x 0.46 - 0.70  -- 48/30
mult: 0.08 x 0.00 - 0.70  -- 56/35
mult: 0.89 x 0.00 - 0.70  -- 64/40
mult: 0.00 x 0.00 - 0.70  -- 72/45
mult: 0.18 x 0.00 - 0.70  -- 80/50
mult: 0.00 x 0.00 - 0.70  -- 88/55
mult: 0.46 x 0.00 - 0.70  -- 96/60
mult: 0.00 x 0.00 - 0.70  -- 104/65
mult: 0.00 x 0.19 - 0.70  -- 112/70
mult: 0.00 x 0.73 - 0.70  -- 120/75

In [100]:
resdatat

array([0.6996368 , 1.8848312 , 0.16573755, 1.130233  , 0.49532753,
       0.60025734, 0.17482497, 0.7240762 , 0.15550727, 0.18735047,
       0.51614475, 0.31159392, 0.9677906 , 0.7728439 , 0.30367106,
       0.27932334, 1.6236207 , 1.4160932 , 1.40742   , 1.1950791 ,
       0.8187757 , 1.2898692 , 0.47157714, 0.99110806, 0.3340667 ,
       0.30414024, 0.7711447 , 0.32051137, 0.1351769 , 0.9285497 ,
       0.544282  , 0.09674796, 0.61331564, 0.29892322, 0.        ,
       0.6311476 , 0.01893103, 0.        , 0.7309462 , 0.8390946 ],
      dtype=float32)

In [101]:
rescolst

array([0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5,
       6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7], dtype=uint32)

In [102]:
resnnzst

array([8, 8, 8, 8, 8], dtype=uint32)

In [103]:
res_np = to_dense(resdatat, rescolst, resnnzst, a.shape[0], mult.T.shape)
res_np.T

IndexError: index 5 is out of bounds for axis 0 with size 5

In [ ]:
(res_np.T-mult).sum()

In [ ]:
res_np.shape

In [ ]:
a.dot(b)

In [ ]:
mult - res_np.T

In [ ]:
res_np.T==mult

# Matmult Dense Transposed2

In [ ]:
b_buf2 = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // multilplies x by y WITH Y TRANSPOSED INDEXING
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize,
                          uint osize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      // osize = get_global_size(1);
      int gidx = get_global_id(0); // col
      // int gidy = get_global_id(1); // row

      for (uint gidy = 0; gidy < osize; gidy++) {
        float ret = 0.0;
        for (uint i = 0; i < msize; i++) {
          ret += x[gidx*msize+i]*y[i*osize+gidy];
          if (gidx==0 && gidy==0)
            printf("\\nmult: %.2f x %.2f - %.2f", x[gidx*msize+i],y[i*msize+gidy], ret);
        }

        //if (gidx==0&&gidy==0)
        //  printf("\\nsum:%.2f", ret);
        res[gidx * osize + gidy] = ret;
      }
    }""").build()

In [ ]:
a.shape, b.shape

In [ ]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [ ]:
rows = a.shape[0]

In [ ]:
mult = mult.astype(np.float32)

In [ ]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [rows], None, a_buf, b_buf2, res_buf, np.uint32(a.shape[1]), np.uint32(b.shape[1]))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)

In [ ]:
(res_np-mult).sum()

In [ ]:
a

In [ ]:
b.T

In [ ]:
a[0]

In [ ]:
b.T[0]

In [ ]:
res_buf

In [ ]:
res_np

In [ ]:
a.dot(b)

In [ ]:
res_np==mult

In [ ]:
res_np.shape

In [ ]:
mult.shape

## Weight update kernel

In [ ]:
bs = 4

In [ ]:
a

In [ ]:
dim = 8

x = np.random.rand(bs,dim).astype(np.float32)
y = np.random.rand(bs,dim).astype(np.float32)
x.shape,y.shape, topk

x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             __global  float* xout,    // INPUT
                             uint topk,
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      uint bs = get_global_size(1);
      uint gid2 = get_global_id(1);

      uint idx = n*gid2+gid;

      float valx = x[idx];
      float valy = y[idx];
      uint posx = 0;
      uint posy = 0;
      for (uint i = 0; i < n; i++) {
        uint idx2 = n*gid2+i;
        float tempval = x[idx2];
        float tempval2 = y[idx2];
        bool larger = tempval > valx;
        bool larger2 = tempval2 > valy;

        barrier(CLK_GLOBAL_MEM_FENCE);
        posx += (larger)?1:0;
        posy += (larger2)?1:0;
        barrier(CLK_GLOBAL_MEM_FENCE);
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
      //printf("posx:%i", posx);
      if (posx < topk) {
        xoutidx[posx+topk*gid2] = gid;
      }
      if (posy < topk) {
        youtidx[posy+topk*gid2] = gid;
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
      if (gid < topk) {
        for (uint j=0; j<topk; j++) {
          float res = x[xoutidx[gid+topk*gid2]+gid2*n] * y[youtidx[j+topk*gid2]+gid2*n];
          //printf("\\nJ:%i  gid:%i", j, gid);
          //printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid+topk*gid2], youtidx[j+topk*gid2], x[xoutidx[gid+topk*gid2]+gid2*n], y[youtidx[j+topk*gid2]+gid2*n]);
          barrier(CLK_GLOBAL_MEM_FENCE);
          xout[gid2*topk*topk+j*topk+gid] = res;
          barrier(CLK_GLOBAL_MEM_FENCE);
          
        }
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
    }""").build()

In [ ]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
#x_cp_buft = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
#x_idx_buft = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
#y_idx_buft = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             __global  float* xout,    // INPUT
                             uint topk,
                             uint bs,
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      //uint bs = get_global_size(1);
      //uint gid2 = get_global_id(1);

      for (uint gid2=0; gid2<bs; gid2++){
        uint idx = n*gid2+gid;

        float valx = x[idx];
        float valy = y[idx];
        uint posx = 0;
        uint posy = 0;
        for (uint i = 0; i < n; i++) {
          uint idx2 = n*gid2+i;
          float tempval = x[idx2];
          float tempval2 = y[idx2];
          bool larger = tempval > valx;
          bool larger2 = tempval2 > valy;

          posx += (larger)?1:0;
          posy += (larger2)?1:0;
        }
        //printf("posx:%i", posx);
        if (posx < topk) {
        xoutidx[posx+topk*gid2] = gid;
        }
        if (posy < topk) {
          youtidx[posy+topk*gid2] = gid;
        }
      }
      for (uint gid2=0; gid2<bs; gid2++){
        if (gid < topk) {
          for (uint j=0; j<topk; j++) {
            float res = x[xoutidx[gid+topk*gid2]+gid2*n] * y[youtidx[j+topk*gid2]+gid2*n];
            //printf("\\nJ:%i  gid:%i", j, gid);
            //printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid+topk*gid2], youtidx[j+topk*gid2], x[xoutidx[gid+topk*gid2]+gid2*n], y[youtidx[j+topk*gid2]+gid2*n]);
            //barrier(CLK_GLOBAL_MEM_FENCE);
            xout[gid2*topk*topk+j*topk+gid] = res;
          }
        }
      }
    }""").build()

In [ ]:
knl = prg.genwupdate2  # Use this Kernel object for repeated calls
evt = knl(queue, [dim], None, x_buf, y_buf, x_cp_buf, np.uint32(topk), np.uint32(bs), x_idx_buf, y_idx_buf)

#evt.wait()
resx = np.zeros(bs*topk*topk).astype(np.float32)
resxidx = np.zeros(bs*topk).astype(np.uint32)
resyidx = np.zeros(bs*topk).astype(np.uint32)

cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)

knl(queue, [dim], None, y_buf, x_buf, x_cp_buft, np.uint32(topk), np.uint32(bs), x_idx_buft, y_idx_buft)

#evt.wait()
resx = np.zeros(bs*topk*topk).astype(np.float32)
resxidx = np.zeros(bs*topk).astype(np.uint32)
resyidx = np.zeros(bs*topk).astype(np.uint32)

cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [ ]:
x

In [ ]:
y

In [104]:
x.shape, y.shape

NameError: name 'y' is not defined

In [ ]:
resx

In [ ]:
resx.reshape(bs,topk,topk)

In [ ]:
resxidx

In [ ]:
resyidx

In [ ]:
idx = 1
xy0 = x[idx].reshape(dim,1)*y[idx]
xy0.shape

In [ ]:
xy0[3][7]

## Weight update kernel new

In [105]:
b

array([[0.49395892, 0.        , 0.46451807, 0.        , 0.        ],
       [0.65864426, 0.        , 0.9786625 , 0.        , 0.        ],
       [0.31044865, 0.        , 0.02502257, 0.        , 0.        ],
       [0.        , 0.46765924, 0.        , 0.        , 0.03160795],
       [0.71082693, 0.        , 0.58997136, 0.        , 0.        ],
       [0.        , 0.        , 0.84622455, 0.5916316 , 0.        ],
       [0.46450275, 0.        , 0.        , 0.        , 0.9429863 ],
       [0.        , 0.34248227, 0.7672639 , 0.        , 0.        ],
       [0.        , 0.        , 0.9855889 , 0.        , 0.2535647 ],
       [0.        , 0.7705159 , 0.31522992, 0.        , 0.        ],
       [0.        , 0.7167741 , 0.        , 0.8238369 , 0.        ],
       [0.        , 0.91970533, 0.        , 0.7873889 , 0.        ],
       [0.        , 0.        , 0.        , 0.404568  , 0.841323  ],
       [0.        , 0.        , 0.674503  , 0.48820347, 0.        ],
       [0.18725161, 0.11607183, 0.

In [106]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [107]:
at

array([[0.        , 0.9562663 , 0.        , 0.        , 0.        ,
        0.11415514, 0.28872353, 0.67626965],
       [0.68347037, 0.5531271 , 0.20103766, 0.        , 0.        ,
        0.82574075, 0.04889954, 0.        ],
       [0.8035886 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.14296253, 0.59893256,
        0.        , 0.6493426 , 0.35563806],
       [0.        , 0.5375557 , 0.        , 0.9103574 , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.74341315, 0.03242496, 0.        ,
        0.27978534, 0.33329687, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.69630474],
       [0.08023349, 0.        , 0.        , 0.71459514, 0.9217044 ,
        0.25018668, 0.        , 0.        ],
       [0.8858316 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.      

In [108]:
c

array([[0.        , 0.95626628, 0.        , 0.        , 0.        ,
        0.11415514, 0.28872353, 0.67626965],
       [0.68347037, 0.55312711, 0.20103766, 0.        , 0.        ,
        0.82574075, 0.04889954, 0.        ],
       [0.80358863, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.14296253, 0.59893256,
        0.        , 0.6493426 , 0.35563806],
       [0.        , 0.53755569, 0.        , 0.91035742, 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.74341315, 0.03242496, 0.        ,
        0.27978534, 0.33329687, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.69630474],
       [0.08023349, 0.        , 0.        , 0.71459514, 0.92170441,
        0.25018668, 0.        , 0.        ],
       [0.88583159, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.      

a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate3(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             uint topk,
                             uint msize,
                             __global  float* xout,    // INPUT
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      uint bs = get_global_size(1);
      uint gid2 = get_global_id(1);



      uint idx = n*gid2+gid;
      float valx = x[idx];
      uint posx = 0;
      for (uint i = 0; i < n; i++) {
        uint idx2 = n*gid2+i;
        float tempval = x[idx2];
        bool larger = tempval > valx;
        posx += (larger)?1:0;
      }
      
      uint idxy = n*gid2+gid;
      float valy = y[idx];
      uint posy = 0;
      for (uint i = 0; i < n; i++) {
        uint idx2 = n*gid2+i;
        float tempval2 = y[idx2];
        bool larger2 = tempval2 > valy;
        posy += (larger2)?1:0;
      }
      
      if (posx < topk) {
        xoutidx[posx+topk*gid2] = idx;
      }
      if (posy < topk) {
        youtidx[posy+topk*gid2] = idxy;
      }
      return;
      if (gid < topk) {
        for (uint j=0; j<topk; j++) {
          float res = x[xoutidx[gid+topk*gid2]+gid2*msize] * y[youtidx[j+topk*gid2]+gid2*msize];
          printf("\\nJ:%i  gid:(%i,%i)", j, gid, gid2);
          printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid+topk*gid2], youtidx[j+topk*gid2], x[xoutidx[gid+topk*gid2]+gid2*n], y[youtidx[j+topk*gid2]+gid2*n]);
          //barrier(CLK_GLOBAL_MEM_FENCE);
          xout[gid2*topk*topk+j*topk+gid] = res;
        }
      }
    }""").build()

In [109]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
x_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, a.shape[0]*4)
y_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.shape[1]*4)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate3(__global  float* x,     // INPUT MATRIX DATA
                              __global  float* y,    // INPUT
                              __global  float* xsum,    // INPUT
                              __global  float* ysum,    // INPUT
                              uint isize,
                              uint msize,
                              uint osize,
                              uint topk,
                              __global  float* xout,
                              __global  uint* xoutidx,   
                              __global  uint* youtidx    
                              ) { 
      uint gid = get_global_id(0);
      
      // get for a: sum axis0  b: sum axis1 then get topk
      ///////////////////////////////////////////////////
      if (gid < isize) {
        xsum[gid] = 0;
        for (uint i=0; i<msize; i++) {
          float val = x[i*isize+gid];
          //if (gid == 0) {
          //  printf("\\nADD VALx: %.2f - %i", val, i*msize+gid);
          //}
          xsum[gid] += val;
        }
        
        float valx = xsum[gid];
        uint posx = 0;
        for (uint i = 0; i < isize; i++) {
          float tempval = xsum[i];
          bool larger = tempval > valx;
          posx += (larger)?1:0;
        }
        if (posx < topk) {
          xoutidx[posx] = gid;
        }
      }
      
      if (gid < osize) {
        ysum[gid] = 0;
        for (uint i=0; i<msize; i++) {
          float val = y[i*osize+gid];
          //if (gid == 0) {
          //  printf("\\nADD VALx: %.2f - %i", val, gid*osize+i);
          //}
          ysum[gid] += val;
        }
        
        float valy = ysum[gid];
        uint posy = 0;
        for (uint i = 0; i < osize; i++) {
          float tempval = ysum[i];
          bool larger = tempval > valy;
          posy += (larger)?1:0;
        }
      
        if (posy < topk) {
          youtidx[posy] = gid;
        }
      }
      
      if (gid < topk) {
        float valx = xoutidx[gid];
        uint posx = 0;
        for (uint i = 0; i < topk; i++) {
          float tempval = xoutidx[i];
          bool larger = tempval < valx;
          posx += (larger)?1:0;
        }
        xoutidx[posx] = valx;
        
        float valy = youtidx[gid];
        uint posy = 0;
        for (uint i = 0; i < topk; i++) {
          float tempval = youtidx[i];
          bool larger = tempval < valy;
          posy += (larger)?1:0;
        }
        youtidx[posy] = valy;
      }
      
      // only calc matrix multiplications for used grads
      ///////////////////////////////////////////////////
      if (gid < topk) {
        uint idxx = xoutidx[gid];
        for (uint j=0; j<topk; j++) {
          uint idxy = youtidx[j];
          xout[j*topk+gid] = 0;
          for (uint k=0; k<msize; k++) {
            uint xidx2 = isize*k+idxx;
            uint yidx2 = osize*k+idxy;
            xout[j*topk+gid] += x[xidx2] * y[yidx2];
            //if (gid == 0 && j == 1)
            //  printf("\\n ADD VAL:%.2f,%.2f - (%i,%i) - (%i,%i,%i)", x[xidx2], y[yidx2], idxx, idxy, gid, j, k);
          }
        }
      }
    }""").build()

In [110]:
a.shape, b.shape

((8, 16), (16, 5))

In [111]:
rows = a.shape[0]
msize = a.shape[1]

In [112]:
cols = b.shape[1]

In [113]:
mult = a.dot(b)

In [114]:
mult = mult.astype(np.float32)

In [115]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.genwupdate3  # Use this Kernel object for repeated calls
evt = knl(queue, [max(rows,cols)], None, a_buf, b_buf, x_sum_buf, y_sum_buf, np.uint32(rows), np.uint32(msize),np.uint32(cols), np.uint32(topk), x_cp_buf, x_idx_buf, y_idx_buf)

resxsum = np.zeros(a.shape[0]).astype(np.float32)
resysum = np.zeros(b.shape[1]).astype(np.float32)
resx = np.zeros(topk*topk).astype(np.float32)
resxidx = np.zeros(topk).astype(np.uint32)
resyidx = np.zeros(topk).astype(np.uint32)

cl.enqueue_copy(queue, resxsum, x_sum_buf)
cl.enqueue_copy(queue, resysum, y_sum_buf)
cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [116]:
resx.reshape(topk,topk)

array([[0.6996368 , 1.8848312 , 1.130233  , 0.49532753, 0.7240762 ],
       [0.15550727, 0.18735047, 0.31159392, 0.9677906 , 0.27932334],
       [1.6236207 , 1.4160932 , 1.1950791 , 0.8187757 , 0.99110806],
       [0.3340667 , 0.30414024, 0.32051137, 0.1351769 , 0.09674796],
       [0.61331564, 0.29892322, 0.6311476 , 0.01893103, 0.8390946 ]],
      dtype=float32)

In [117]:
resxsum

array([3.0937524, 3.5246177, 2.3468003, 3.2115116, 3.102983 , 2.4004393,
       2.1758807, 2.6182923], dtype=float32)

In [118]:
resysum

array([3.5506585, 3.3332088, 5.77045  , 3.095629 , 2.0694818],
      dtype=float32)

In [119]:
a.sum(axis=1)

array([3.0937524, 3.5246177, 2.3468003, 3.2115116, 3.102983 , 2.4004393,
       2.1758807, 2.6182923], dtype=float32)

In [120]:
b.sum(axis=0)

array([3.5506585, 3.3332088, 5.77045  , 3.095629 , 2.0694818],
      dtype=float32)

In [121]:
mult

array([[0.6996368 , 0.15550727, 1.6236207 , 0.3340667 , 0.61331564],
       [1.8848312 , 0.18735047, 1.4160932 , 0.30414024, 0.29892322],
       [0.16573755, 0.51614475, 1.40742   , 0.7711447 , 0.        ],
       [1.130233  , 0.31159392, 1.1950791 , 0.32051137, 0.6311476 ],
       [0.49532753, 0.9677906 , 0.8187757 , 0.1351769 , 0.01893103],
       [0.60025734, 0.7728439 , 1.2898692 , 0.9285497 , 0.        ],
       [0.17482497, 0.30367106, 0.47157714, 0.544282  , 0.7309462 ],
       [0.7240762 , 0.27932334, 0.99110806, 0.09674796, 0.8390946 ]],
      dtype=float32)

In [122]:
resxidx

array([0, 1, 3, 4, 7], dtype=uint32)

In [123]:
resyidx

array([0, 1, 2, 3, 4], dtype=uint32)

In [124]:
idx = 1
xy0 = x[idx].reshape(dim,1)*y[idx]
xy0.shape

NameError: name 'dim' is not defined

In [ ]:
xy0[0][0]

## Weight update kernel new2 (sparse ouput)

In [ ]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [ ]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
x_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, a.shape[0]*4)
y_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.shape[1]*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topkx*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topky*4)
sdata_buf = cl.Buffer(ctx, mf.READ_WRITE, a.shape[0]*topkx*4)
sidxs_buf = cl.Buffer(ctx, mf.READ_WRITE, a.shape[0]*topkx*4)
snnzs_buf = cl.Buffer(ctx, mf.READ_WRITE, a.shape[0]*4)
sdatat_buf = cl.Buffer(ctx, mf.READ_WRITE, b.shape[1]*topky*4)
sidxst_buf = cl.Buffer(ctx, mf.READ_WRITE, b.shape[1]*topky*4)
snnzst_buf = cl.Buffer(ctx, mf.READ_WRITE, b.shape[1]*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate4(__global  float* x,     // INPUT MATRIX DATA
                              __global  float* y,    // INPUT
                              __global  float* xsum,    // INPUT
                              __global  float* ysum,    // INPUT
                              uint isize,
                              uint msize,
                              uint osize,
                              uint topkx,
                              uint topky,
                              __global  uint*  xoutidx,
                              __global  uint*  youtidx,
                              __global  float* matData,     // OUTPUT MATRIX DATA
                              __global  uint*  colIdx,
                              __global  uint*  rowNnz,
                              __global  float* matDatat,    // OUTPUT MATRIX DATA
                              __global  uint*  colIdxt,
                              __global  uint*  rowNnzt
                              ) {
      uint gid = get_global_id(0);

      // get for a: sum axis0  b: sum axis1 then get topk
      ///////////////////////////////////////////////////
      if (gid < isize) {
        xsum[gid] = 0;
        for (uint i=0; i<msize; i++) {
          float val = x[i*isize+gid];
          //if (gid == 0) {
          //  printf("\\nADD VALx: %.2f - %i", val, i*msize+gid);
          //}
          xsum[gid] += val;
        }

        float valx = xsum[gid];
        uint posx = 0;
        for (uint i = 0; i < isize; i++) {
          float tempval = fabs(xsum[i]);
          bool larger = tempval > fabs(valx);
          posx += (larger)?1:0;
        }
        if (posx < topky) {
          youtidx[posx] = gid;
        }
      }

      if (gid < osize) {
        ysum[gid] = 0;
        for (uint i=0; i<msize; i++) {
          float val = y[i*osize+gid];
          //if (gid == 0) {
          //  printf("\\nADD VALx: %.2f - %i", val, gid*osize+i);
          //}
          ysum[gid] += val;
        }

        float valy = ysum[gid];
        uint posy = 0;
        for (uint i = 0; i < osize; i++) {
          float tempval = fabs(ysum[i]);
          bool larger = tempval > fabs(valy);
          posy += (larger)?1:0;
        }

        if (posy < topkx) {
          xoutidx[posy] = gid;
        }
      }

      if (gid < topkx) {
        float valx = xoutidx[gid];
        uint posx = 0;
        for (uint i = 0; i < topkx; i++) {
          float tempval = xoutidx[i];
          bool larger = tempval < valx;
          posx += (larger)?1:0;
        }
        xoutidx[gid] = gid;
      }

      if (gid < topky) {
        float valy = youtidx[gid];
        uint posy = 0;
        for (uint i = 0; i < topky; i++) {
          float tempval = youtidx[i];
          bool larger = tempval < valy;
          posy += (larger)?1:0;
        }
        youtidx[gid] = gid;
      }

      // only calc matrix multiplications for used grads
      ///////////////////////////////////////////////////
      if (gid < isize) {
        for (uint i=0; i<topkx; i++) {
          matData[gid*topkx+i] = 0;
          colIdx[gid*topkx+i] = 0;
        }
        rowNnz[gid] = 0;
      }
      if (gid < osize) {
        for (uint i=0; i<topky; i++) {
          matDatat[gid*topky+i] = 0;
          colIdxt[gid*topky+i] = 0;
        }
        rowNnzt[gid] = 0;
      }


      if (gid < topkx) {
        uint idxx = xoutidx[gid];
        for (uint j=0; j<topky; j++) {
          uint idxy = youtidx[j];
          //printf("\\nIDXX:%i  IDXY:%i", idxx, idxy);
          for (uint k=0; k<msize; k++) {
            uint xidx2 = isize*k+idxy;
            uint yidx2 = osize*k+idxx;
            uint colidx = idxy;
            matDatat[idxx*topky+j] += x[xidx2] * y[yidx2];
            colIdxt[idxx*topky+j] = idxy;
            if (gid == 0)
              printf("\\n ADD VAL:%.2f,%.2f - (%i,%i) - (%i,%i,%i)", x[xidx2], y[yidx2], idxx, idxy, gid, j, k);
          }
          rowNnzt[idxx] += 1;
        }
      }
      if (gid < topky) {
        uint idxx = youtidx[gid];
        for (uint j=0; j<topkx; j++) {
          uint idxy = xoutidx[j];
          //printf("\\nIDXX:%i  IDXY:%i", idxx, idxy);
          for (uint k=0; k<msize; k++) {
            uint xidx2 = isize*k+idxx;
            uint yidx2 = osize*k+idxy;
            uint colidx = idxy;
            matData[idxx*topkx+j] += x[xidx2] * y[yidx2];
            colIdx[idxx*topkx+j] = idxy;
            if (gid == 0)
              printf("\\n ADD VAL:%.2f,%.2f - (%i,%i) - (%i,%i,%i)", x[xidx2], y[yidx2], idxx, idxy, gid, j, k);
          }
          rowNnz[idxx] += 1;
        }
      }
    }""").build()

In [ ]:
a.shape, b.shape

In [ ]:
rows = a.shape[0]
msize = a.shape[1]

In [ ]:
cols = b.shape[1]

In [ ]:
mult = a.dot(b)

In [ ]:
mult = mult.astype(np.float32)

In [ ]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.genwupdate4  # Use this Kernel object for repeate/duald calls
evt = knl(queue, [max(rows,cols)], None, a_buf, b_buf, x_sum_buf, y_sum_buf, np.uint32(rows), np.uint32(msize),np.uint32(cols), 
          np.uint32(topkx),np.uint32(topky), x_idx_buf, y_idx_buf, sdata_buf, sidxs_buf, snnzs_buf, sdatat_buf, sidxst_buf, snnzst_buf)

In [ ]:
resxsum = np.zeros(a.shape[0]).astype(np.float32)
resysum = np.zeros(b.shape[1]).astype(np.float32)
resxidx = np.zeros(topkx).astype(np.uint32)
resyidx = np.zeros(topky).astype(np.uint32)
resxdat = np.zeros(a.shape[0]*topkx).astype(np.float32)
resxcol = np.zeros(a.shape[0]*topkx).astype(np.uint32)
resxnnz = np.zeros(a.shape[0]).astype(np.uint32)
resxdatt = np.zeros(b.shape[1]*topky).astype(np.float32)
resxcolt = np.zeros(b.shape[1]*topky).astype(np.uint32)
resxnnzt = np.zeros(b.shape[1]).astype(np.uint32)

cl.enqueue_copy(queue, resxsum, x_sum_buf)
cl.enqueue_copy(queue, resysum, y_sum_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)
cl.enqueue_copy(queue, resxdat, sdata_buf)
cl.enqueue_copy(queue, resxcol, sidxs_buf)
cl.enqueue_copy(queue, resxnnz, snnzs_buf)
cl.enqueue_copy(queue, resxdatt, sdatat_buf)
cl.enqueue_copy(queue, resxcolt, sidxst_buf)
cl.enqueue_copy(queue, resxnnzt, snnzst_buf)

## results

In [ ]:
topkx, topky

In [ ]:
mult.shape

In [ ]:
mult

In [ ]:
resxdatt.reshape(b.shape[1],topky).T

In [ ]:
resxcol.reshape(a.shape[0],topkx)

In [ ]:
resxnnz.reshape(a.shape[0])

In [ ]:
resxdatt.reshape(b.shape[1],topky)

In [ ]:
resxcolt.reshape(b.shape[1],topky)

In [ ]:
resxnnzt.reshape(b.shape[1])

In [125]:
resdense = to_dense(resxdat, resxcol, resxnnz, topkx, mult.shape)
resdense

NameError: name 'resxdat' is not defined

In [ ]:
resdenset = to_dense(resxdatt, resxcolt, resxnnzt, topky, mult.T.shape)
resdenset

In [ ]:
resdense == resdenset.T

## comp

In [ ]:
resxsum

In [ ]:
a.sum(axis=1)

In [126]:
resysum

array([3.5506585, 3.3332088, 5.77045  , 3.095629 , 2.0694818],
      dtype=float32)

In [127]:
b.sum(axis=0)

array([3.5506585, 3.3332088, 5.77045  , 3.095629 , 2.0694818],
      dtype=float32)

In [128]:
mult

array([[0.6996368 , 0.15550727, 1.6236207 , 0.3340667 , 0.61331564],
       [1.8848312 , 0.18735047, 1.4160932 , 0.30414024, 0.29892322],
       [0.16573755, 0.51614475, 1.40742   , 0.7711447 , 0.        ],
       [1.130233  , 0.31159392, 1.1950791 , 0.32051137, 0.6311476 ],
       [0.49532753, 0.9677906 , 0.8187757 , 0.1351769 , 0.01893103],
       [0.60025734, 0.7728439 , 1.2898692 , 0.9285497 , 0.        ],
       [0.17482497, 0.30367106, 0.47157714, 0.544282  , 0.7309462 ],
       [0.7240762 , 0.27932334, 0.99110806, 0.09674796, 0.8390946 ]],
      dtype=float32)

In [129]:
resxidx

array([0, 1, 3, 4, 7], dtype=uint32)

In [130]:
resyidx

array([0, 1, 2, 3, 4], dtype=uint32)

In [131]:
asdf

NameError: name 'asdf' is not defined

## Weight update kernel new3 (sparse ouput)

In [ ]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [ ]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
x_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, a.shape[0]*4)
y_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.shape[1]*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topkx*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topky*4)
sdata_buf = cl.Buffer(ctx, mf.READ_WRITE, a.shape[0]*topkx*4)
sidxs_buf = cl.Buffer(ctx, mf.READ_WRITE, a.shape[0]*topkx*4)
snnzs_buf = cl.Buffer(ctx, mf.READ_WRITE, a.shape[0]*4)
sdatat_buf = cl.Buffer(ctx, mf.READ_WRITE, b.shape[1]*topky*4)
sidxst_buf = cl.Buffer(ctx, mf.READ_WRITE, b.shape[1]*topky*4)
snnzst_buf = cl.Buffer(ctx, mf.READ_WRITE, b.shape[1]*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate4(__global  float* x,     // INPUT MATRIX DATA
                              __global  float* y,    // INPUT
                              __global  float* xsum,    // INPUT
                              __global  float* ysum,    // INPUT
                              uint isize,
                              uint msize,
                              uint osize,
                              uint topkx,
                              uint topky,
                              __global  uint*  xoutidx,
                              __global  uint*  youtidx,
                              __global  float* matData,     // OUTPUT MATRIX DATA
                              __global  uint*  colIdx,
                              __global  uint*  rowNnz,
                              __global  float* matDatat,    // OUTPUT MATRIX DATA
                              __global  uint*  colIdxt,
                              __global  uint*  rowNnzt
                              ) {
      uint gid = get_global_id(0);

      // get for a: sum axis0  b: sum axis1 then get topk
      ///////////////////////////////////////////////////
      if (gid < isize) {
        xsum[gid] = 0;
        for (uint i=0; i<msize; i++) {
          float val = x[i*isize+gid];
          //if (gid == 0) {
          //  printf("\\nADD VALx: %.2f - %i", val, i*msize+gid);
          //}
          xsum[gid] += val;
        }

        float valx = xsum[gid];
        uint posx = 0;
        for (uint i = 0; i < isize; i++) {
          float tempval = fabs(xsum[i]);
          bool larger = tempval > fabs(valx);
          posx += (larger)?1:0;
        }
        if (posx < topky) {
          youtidx[posx] = gid;
        }
      }

      if (gid < osize) {
        ysum[gid] = 0;
        for (uint i=0; i<msize; i++) {
          float val = y[i*osize+gid];
          //if (gid == 0) {
          //  printf("\\nADD VALx: %.2f - %i", val, gid*osize+i);
          //}
          ysum[gid] += val;
        }

        float valy = ysum[gid];
        uint posy = 0;
        for (uint i = 0; i < osize; i++) {
          float tempval = fabs(ysum[i]);
          bool larger = tempval > fabs(valy);
          posy += (larger)?1:0;
        }

        if (posy < topkx) {
          xoutidx[posy] = gid;
        }
      }

      if (gid < topkx) {
        float valx = xoutidx[gid];
        uint posx = 0;
        for (uint i = 0; i < topkx; i++) {
          float tempval = xoutidx[i];
          bool larger = tempval < valx;
          posx += (larger)?1:0;
        }
        xoutidx[gid] = gid;
      }

      if (gid < topky) {
        float valy = youtidx[gid];
        uint posy = 0;
        for (uint i = 0; i < topky; i++) {
          float tempval = youtidx[i];
          bool larger = tempval < valy;
          posy += (larger)?1:0;
        }
        youtidx[gid] = gid;
      }

      // only calc matrix multiplications for used grads
      ///////////////////////////////////////////////////
      if (gid < isize) {
        for (uint i=0; i<topkx; i++) {
          matData[gid*topkx+i] = 0;
          colIdx[gid*topkx+i] = 0;
        }
        rowNnz[gid] = 0;
      }
      if (gid < osize) {
        for (uint i=0; i<topky; i++) {
          matDatat[gid*topky+i] = 0;
          colIdxt[gid*topky+i] = 0;
        }
        rowNnzt[gid] = 0;
      }


      if (gid < topkx) {
        uint idxx = xoutidx[gid];
        for (uint j=0; j<topky; j++) {
          uint idxy = youtidx[j];
          //printf("\\nIDXX:%i  IDXY:%i", idxx, idxy);
          for (uint k=0; k<msize; k++) {
            uint xidx2 = isize*k+idxy;
            uint yidx2 = osize*k+idxx;
            uint colidx = idxy;
            matDatat[idxx*topky+j] += x[xidx2] * y[yidx2];
            colIdxt[idxx*topky+j] = idxy;
            if (gid == 0)
              printf("\\n ADD VAL:%.2f,%.2f - (%i,%i) - (%i,%i,%i)", x[xidx2], y[yidx2], idxx, idxy, gid, j, k);
          }
          rowNnzt[idxx] += 1;
        }
      }
      if (gid < topky) {
        uint idxx = youtidx[gid];
        for (uint j=0; j<topkx; j++) {
          uint idxy = xoutidx[j];
          //printf("\\nIDXX:%i  IDXY:%i", idxx, idxy);
          for (uint k=0; k<msize; k++) {
            uint xidx2 = isize*k+idxx;
            uint yidx2 = osize*k+idxy;
            uint colidx = idxy;
            matData[idxx*topkx+j] += x[xidx2] * y[yidx2];
            colIdx[idxx*topkx+j] = idxy;
            if (gid == 0)
              printf("\\n ADD VAL:%.2f,%.2f - (%i,%i) - (%i,%i,%i)", x[xidx2], y[yidx2], idxx, idxy, gid, j, k);
          }
          rowNnz[idxx] += 1;
        }
      }
    }""").build()

In [ ]:
a.shape, b.shape

In [ ]:
rows = a.shape[0]
msize = a.shape[1]

In [ ]:
cols = b.shape[1]

In [ ]:
mult = a.dot(b)

In [132]:
mult = mult.astype(np.float32)

In [133]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.genwupdate4  # Use this Kernel object for repeate/duald calls
evt = knl(queue, [max(rows,cols)], None, a_buf, b_buf, x_sum_buf, y_sum_buf, np.uint32(rows), np.uint32(msize),np.uint32(cols), 
          np.uint32(topkx),np.uint32(topky), x_idx_buf, y_idx_buf, sdata_buf, sidxs_buf, snnzs_buf, sdatat_buf, sidxst_buf, snnzst_buf)

AttributeError: 'genwupdate4' was not found as a program info attribute or as a kernel name

In [ ]:
resxsum = np.zeros(a.shape[0]).astype(np.float32)
resysum = np.zeros(b.shape[1]).astype(np.float32)
resxidx = np.zeros(topkx).astype(np.uint32)
resyidx = np.zeros(topky).astype(np.uint32)
resxdat = np.zeros(a.shape[0]*topkx).astype(np.float32)
resxcol = np.zeros(a.shape[0]*topkx).astype(np.uint32)
resxnnz = np.zeros(a.shape[0]).astype(np.uint32)
resxdatt = np.zeros(b.shape[1]*topky).astype(np.float32)
resxcolt = np.zeros(b.shape[1]*topky).astype(np.uint32)
resxnnzt = np.zeros(b.shape[1]).astype(np.uint32)

cl.enqueue_copy(queue, resxsum, x_sum_buf)
cl.enqueue_copy(queue, resysum, y_sum_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)
cl.enqueue_copy(queue, resxdat, sdata_buf)
cl.enqueue_copy(queue, resxcol, sidxs_buf)
cl.enqueue_copy(queue, resxnnz, snnzs_buf)
cl.enqueue_copy(queue, resxdatt, sdatat_buf)
cl.enqueue_copy(queue, resxcolt, sidxst_buf)
cl.enqueue_copy(queue, resxnnzt, snnzst_buf)

## results

In [ ]:
topkx, topky

In [ ]:
mult.shape

In [ ]:
mult

In [134]:
resxdatt.reshape(b.shape[1],topky).T

NameError: name 'resxdatt' is not defined

In [ ]:
resxcol.reshape(a.shape[0],topkx)

In [ ]:
resxnnz.reshape(a.shape[0])

In [135]:
resxdatt.reshape(b.shape[1],topky)

NameError: name 'resxdatt' is not defined

In [ ]:
resxcolt.reshape(b.shape[1],topky)

In [ ]:
resxnnzt.reshape(b.shape[1])

In [136]:
resdense = to_dense(resxdat, resxcol, resxnnz, topkx, mult.shape)
resdense

NameError: name 'resxdat' is not defined

In [ ]:
resdenset = to_dense(resxdatt, resxcolt, resxnnzt, topky, mult.T.shape)
resdenset

In [137]:
resdense == resdenset.T

NameError: name 'resdense' is not defined

## comp

In [ ]:
resxsum

In [138]:
a.sum(axis=1)

array([3.0937524, 3.5246177, 2.3468003, 3.2115116, 3.102983 , 2.4004393,
       2.1758807, 2.6182923], dtype=float32)

In [139]:
resysum

array([3.5506585, 3.3332088, 5.77045  , 3.095629 , 2.0694818],
      dtype=float32)

In [140]:
b.sum(axis=0)

array([3.5506585, 3.3332088, 5.77045  , 3.095629 , 2.0694818],
      dtype=float32)

In [141]:
mult

array([[0.6996368 , 0.15550727, 1.6236207 , 0.3340667 , 0.61331564],
       [1.8848312 , 0.18735047, 1.4160932 , 0.30414024, 0.29892322],
       [0.16573755, 0.51614475, 1.40742   , 0.7711447 , 0.        ],
       [1.130233  , 0.31159392, 1.1950791 , 0.32051137, 0.6311476 ],
       [0.49532753, 0.9677906 , 0.8187757 , 0.1351769 , 0.01893103],
       [0.60025734, 0.7728439 , 1.2898692 , 0.9285497 , 0.        ],
       [0.17482497, 0.30367106, 0.47157714, 0.544282  , 0.7309462 ],
       [0.7240762 , 0.27932334, 0.99110806, 0.09674796, 0.8390946 ]],
      dtype=float32)

In [142]:
resxidx

array([0, 1, 3, 4, 7], dtype=uint32)

In [143]:
resyidx

array([0, 1, 2, 3, 4], dtype=uint32)

In [144]:
asdf

NameError: name 'asdf' is not defined

## Prune Weights

In [ ]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)

prg = cl.Program(ctx, """
    // prunes weights smaller than a constant C
    __kernel void prune(__global  float* matData,     // INPUT MATRIX DATA
                        __global  uint*  colIdx,
                        __global  uint*  rowNnz,
                        uint ellw,
                        float pruneval) { 
      uint gid = get_global_id(0);
      
      uint nnzs = rowNnz[gid];
      for (uint i=0; i<nnzs; i++) {
        uint idx = ellw * gid + i;
        float val = matData[idx];
        printf("\\nDATA:%.2f - %.2f", matData[idx], pruneval);
        if(fabs(val)<pruneval) {
          printf("\\nPRUNE(%i): %.2f", gid, matData[idx]);
          for (uint j=i; j<=nnzs-1; j++) {
            uint idx2 = ellw * gid + j;
            matData[idx2] = matData[idx2+1];
            colIdx[idx2] = colIdx[idx2+1];
          }
          matData[ellw*gid+nnzs] = 0;
          colIdx[ellw*gid+nnzs] = 0;
          rowNnz[gid] -= 1;
          nnzs = rowNnz[gid];
        }
      }
    }""").build()

In [ ]:
a.shape

In [ ]:
rows = a.shape[0]
cols = a.shape[1]

pruneval = .35

In [ ]:
knl = prg.prune  # Use this Kernel object for repeated calls
evt = knl(queue, [rows,], None, adata_buf, acols_buf, annzs_buf, np.uint32(ellwa), np.float32(pruneval))

In [145]:
resxdat = np.zeros(adata.shape).astype(np.float32)
resxcol = np.zeros(acols.shape).astype(np.uint32)
resxnnz = np.zeros(annz.shape).astype(np.uint32)

cl.enqueue_copy(queue, resxdat, adata_buf)
cl.enqueue_copy(queue, resxcol, acols_buf)
cl.enqueue_copy(queue, resxnnz, annzs_buf)

In [146]:
a

array([[0.        , 0.68347037, 0.8035886 , 0.        , 0.        ,
        0.        , 0.        , 0.08023349, 0.8858316 , 0.        ,
        0.17861794, 0.        , 0.46201044, 0.        , 0.        ,
        0.        ],
       [0.9562663 , 0.5531271 , 0.        , 0.        , 0.5375557 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.20370705, 0.35530138, 0.        , 0.        ,
        0.9186601 ],
       [0.        , 0.20103766, 0.        , 0.        , 0.        ,
        0.74341315, 0.        , 0.        , 0.        , 0.5787496 ,
        0.        , 0.05387774, 0.        , 0.5917517 , 0.17797045,
        0.        ],
       [0.        , 0.        , 0.        , 0.14296253, 0.9103574 ,
        0.03242496, 0.        , 0.71459514, 0.        , 0.        ,
        0.        , 0.        , 0.7448136 , 0.        , 0.        ,
        0.66635793],
       [0.        , 0.        , 0.        , 0.59893256, 0.        ,
        0.        , 0.        , 

In [147]:
adata.reshape((4,-1))

array([[0.68347037, 0.8035886 , 0.08023349, 0.8858316 , 0.17861794,
        0.46201044, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.9562663 , 0.5531271 , 0.5375557 , 0.20370705,
        0.35530138, 0.9186601 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.20103766, 0.74341315, 0.5787496 , 0.05387774, 0.5917517 ,
        0.17797045, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.14296253, 0.9103574 , 0.03242496, 0.71459514,
        0.7448136 , 0.66635793, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.59893256, 0.9217044 , 0.16333483, 0.17167741, 0.7605819 ,
        0.48675168, 0.        , 0.        , 0.    

In [148]:
acols.reshape((4,-1))

array([[ 1,  2,  7,  8, 10, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  1,  4, 11, 12, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  5,  9, 11, 13, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         3,  4,  5,  7, 12, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 3,  7,  9, 11, 14, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  1,  5,  7, 10, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  3,  5, 12, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  3,  6,  8, 11, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=uint32)

In [149]:
resxdat.reshape((4,-1))

array([[0.68347037, 0.8035886 , 0.08023349, 0.8858316 , 0.17861794,
        0.46201044, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.9562663 , 0.5531271 , 0.5375557 , 0.20370705,
        0.35530138, 0.9186601 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.20103766, 0.74341315, 0.5787496 , 0.05387774, 0.5917517 ,
        0.17797045, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.14296253, 0.9103574 , 0.03242496, 0.71459514,
        0.7448136 , 0.66635793, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.59893256, 0.9217044 , 0.16333483, 0.17167741, 0.7605819 ,
        0.48675168, 0.        , 0.        , 0.    

In [150]:
resxcol.reshape((4,-1))

array([[ 1,  2,  7,  8, 10, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  1,  4, 11, 12, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  5,  9, 11, 13, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         3,  4,  5,  7, 12, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 3,  7,  9, 11, 14, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  1,  5,  7, 10, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  3,  5, 12, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  3,  6,  8, 11, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=uint32)

In [151]:
resxnnz

array([6, 6, 6, 6, 6, 6, 6, 6], dtype=uint32)

## results

In [152]:
mult.T

array([[0.6996368 , 1.8848312 , 0.16573755, 1.130233  , 0.49532753,
        0.60025734, 0.17482497, 0.7240762 ],
       [0.15550727, 0.18735047, 0.51614475, 0.31159392, 0.9677906 ,
        0.7728439 , 0.30367106, 0.27932334],
       [1.6236207 , 1.4160932 , 1.40742   , 1.1950791 , 0.8187757 ,
        1.2898692 , 0.47157714, 0.99110806],
       [0.3340667 , 0.30414024, 0.7711447 , 0.32051137, 0.1351769 ,
        0.9285497 , 0.544282  , 0.09674796],
       [0.61331564, 0.29892322, 0.        , 0.6311476 , 0.01893103,
        0.        , 0.7309462 , 0.8390946 ]], dtype=float32)

In [153]:
resxdat.reshape(a.shape[0],topk)

ValueError: cannot reshape array of size 128 into shape (8,5)

In [ ]:
resxcol.reshape(a.shape[0],topk)

In [ ]:
resxnnz.reshape(a.shape[0])

In [ ]:
resxdatt.reshape(b.shape[1],topk)

In [ ]:
resxcolt.reshape(b.shape[1],topk)

In [ ]:
resxnnzt.reshape(b.shape[1])

### Update Vals (add sparse)

In [ ]:
b.shape

In [154]:
randadd = np.random.rand(*b.shape)

In [155]:
randdata, randcols, randnnz, randellw = to_data(randadd)
bdata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bdata)
bcols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcols)
bnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnz)
randdata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=randdata)
randcols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=randcols)
randnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=randnnz)


prg = cl.Program(ctx, """
    // Every global_id_0 works on a row
    __kernel void adddense(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            float  lr,
                            uint   ellwidth,
                            __global  float* matDataAdd,     // INPUT MATRIX DATA
                            __global  uint*  colIdxAdd,
                            __global  uint*  rowNnzAdd,
                            uint ellwidthAdd
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);

      uint nnz    = rowNnz[gid];

      uint baseidxs = gid*ellwidth;
      uint baseidxd = gid*ellwidthAdd;

      uint nnzadd = rowNnzAdd[gid];
      
      uint m = 0;
      for (uint i=0; i<nnzadd; i++) {
        float addval = matDataAdd[baseidxd+i] * lr;
        uint addcol = colIdxAdd[baseidxd+i];
        
        if (addval == 0.0) {
          //printf("\\nZERO VAL, CONT: %.2f - %i", addval, gid);
          continue;
        }

        uint refcol = colIdx[baseidxs+i];
        m = 0;
        while (refcol < addcol && (i+m) < nnz) {
          m += 1;
          refcol = colIdx[baseidxs+i+m];
        }

        //if (gid == 0)
        //  printf("\\nADD VAL:%.2f  ADDCOL:%i  ref:(%i)  gid/i/m:(%i/%i%i)", addval, addcol, refcol, gid,i,m);
        
        if (addcol == refcol) {
          matData[baseidxs+i+m] += addval;
          //if (gid == 0)
          //  printf("\\nINCREMENT: %.2f",addval);
          continue;
        } else {
          //if (gid == 0)
          //  printf("\\nADD: %.2f %i-%i",addval, addcol, refcol);
          if (rowNnz[gid] >= ellwidth) {
            break;
          }

          for (uint j=nnz; j>i+m; j--) {
            //printf("\\nMOVE:%.2f", matData[baseidx+j-1]);
            colIdx[baseidxs+j] = colIdx[baseidxs+j-1];
            matData[baseidxs+j] = matData[baseidxs+j-1];
          }
          rowNnz[gid] += 1;
          nnz = rowNnz[gid];
        
          //if (gid == 0)
          //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, addcol);
          matData[baseidxs+i+m] = addval;
          colIdx[baseidxs+i+m] = addcol;
        }
      }
    }""").build()

In [156]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [157]:
rows = b.shape[0]

In [158]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)
knl = prg.adddense  # Use this Kernel object for repeated calls
knl(queue, [rows], None, bdata_buf, bcols_buf, bnnzs_buf, np.float32(1), np.uint32(ellwb), 
    randdata_buf, randcols_buf, randnnzs_buf, np.uint32(randellw))

In [159]:
data_res = np.empty_like(bdata)
cols_res = np.empty_like(bcols)
nnzs_res = np.empty_like(bnnz)
cl.enqueue_copy(queue, data_res, bdata_buf, is_blocking=True)
cl.enqueue_copy(queue, cols_res, bcols_buf, is_blocking=True)
cl.enqueue_copy(queue, nnzs_res, bnnzs_buf, is_blocking=True)

In [160]:
b

array([[0.49395892, 0.        , 0.46451807, 0.        , 0.        ],
       [0.65864426, 0.        , 0.9786625 , 0.        , 0.        ],
       [0.31044865, 0.        , 0.02502257, 0.        , 0.        ],
       [0.        , 0.46765924, 0.        , 0.        , 0.03160795],
       [0.71082693, 0.        , 0.58997136, 0.        , 0.        ],
       [0.        , 0.        , 0.84622455, 0.5916316 , 0.        ],
       [0.46450275, 0.        , 0.        , 0.        , 0.9429863 ],
       [0.        , 0.34248227, 0.7672639 , 0.        , 0.        ],
       [0.        , 0.        , 0.9855889 , 0.        , 0.2535647 ],
       [0.        , 0.7705159 , 0.31522992, 0.        , 0.        ],
       [0.        , 0.7167741 , 0.        , 0.8238369 , 0.        ],
       [0.        , 0.91970533, 0.        , 0.7873889 , 0.        ],
       [0.        , 0.        , 0.        , 0.404568  , 0.841323  ],
       [0.        , 0.        , 0.674503  , 0.48820347, 0.        ],
       [0.18725161, 0.11607183, 0.

In [161]:
bcols

array([0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 1, 4, 0, 0, 0, 0, 2,
       0, 0, 0, 2, 3, 0, 0, 0, 0, 4, 0, 0, 0, 1, 2, 0, 0, 0, 2, 4, 0, 0,
       0, 1, 2, 0, 0, 0, 1, 3, 0, 0, 0, 1, 3, 0, 0, 0, 3, 4, 0, 0, 0, 2,
       3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0], dtype=uint32)

In [162]:
data_res

array([0.6709938 , 0.9082271 , 1.3086331 , 0.13836125, 0.37077376,
       0.80051553, 0.2554239 , 1.7198983 , 0.5749973 , 0.58092856,
       0.5559536 , 0.37022075, 0.31031638, 0.04638312, 0.89031464,
       0.4462557 , 0.48636654, 0.49408567, 0.4352037 , 0.26299107,
       1.23404   , 0.98933345, 1.2675607 , 0.9670713 , 0.24639843,
       0.7003624 , 0.7240451 , 1.3544325 , 0.82925963, 0.50561154,
       1.3491886 , 0.9135105 , 0.22866099, 0.52384585, 1.0583676 ,
       0.09020234, 0.6252836 , 1.7359526 , 0.04652401, 0.40306655,
       0.8626351 , 0.16699363, 1.9260955 , 0.85600585, 1.2378309 ,
       0.19067204, 1.7169733 , 1.1091754 , 0.35402992, 0.49338102,
       0.22329962, 1.4974368 , 0.3408471 , 1.0464573 , 0.72643334,
       0.1980827 , 1.5736473 , 0.3903598 , 1.6717314 , 0.9210825 ,
       0.52586645, 0.60021836, 0.9936902 , 0.9282068 , 0.9463449 ,
       0.72692764, 0.9487509 , 0.8637423 , 1.0906332 , 0.00412456,
       0.930763  , 0.3343119 , 0.01742795, 0.73911434, 0.90666

In [163]:
cols_res

array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1,
       2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3,
       4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0,
       1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4], dtype=uint32)

In [164]:
nnzs_res

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], dtype=uint32)

In [165]:
randadd

array([[0.17703491, 0.9082271 , 0.84411504, 0.13836125, 0.37077377],
       [0.14187126, 0.2554239 , 0.74123587, 0.57499728, 0.58092856],
       [0.245505  , 0.37022075, 0.28529381, 0.04638312, 0.89031461],
       [0.4462557 , 0.01870731, 0.49408568, 0.43520371, 0.23138313],
       [0.52321305, 0.98933346, 0.67758944, 0.96707127, 0.24639843],
       [0.7003624 , 0.72404507, 0.50820789, 0.23762805, 0.50561157],
       [0.88468577, 0.91351048, 0.22866098, 0.52384588, 0.11538133],
       [0.09020234, 0.2828013 , 0.96868881, 0.04652401, 0.40306654],
       [0.8626351 , 0.16699363, 0.94050658, 0.85600583, 0.98426617],
       [0.19067204, 0.94645738, 0.79394556, 0.35402994, 0.49338102],
       [0.22329962, 0.7806627 , 0.34084711, 0.22262034, 0.72643334],
       [0.1980827 , 0.65394185, 0.39035979, 0.88434246, 0.92108251],
       [0.52586642, 0.60021833, 0.9936902 , 0.52363882, 0.10502191],
       [0.72692763, 0.94875092, 0.18923926, 0.60242976, 0.00412456],
       [0.74351135, 0.21824009, 0.

In [166]:
adenseadd = to_dense(data_res, cols_res, nnzs_res, ellwb, b.shape)
adenseadd

array([[0.6709938 , 0.90822709, 1.30863309, 0.13836125, 0.37077376],
       [0.80051553, 0.2554239 , 1.71989834, 0.57499731, 0.58092856],
       [0.55595362, 0.37022075, 0.31031638, 0.04638312, 0.89031464],
       [0.44625571, 0.48636654, 0.49408567, 0.4352037 , 0.26299107],
       [1.23404002, 0.98933345, 1.26756072, 0.96707129, 0.24639843],
       [0.70036238, 0.7240451 , 1.35443246, 0.82925963, 0.50561154],
       [1.34918857, 0.9135105 , 0.22866099, 0.52384585, 1.05836761],
       [0.09020234, 0.6252836 , 1.73595262, 0.04652401, 0.40306655],
       [0.86263508, 0.16699363, 1.92609549, 0.85600585, 1.23783088],
       [0.19067204, 1.7169733 , 1.10917544, 0.35402992, 0.49338102],
       [0.22329962, 1.49743676, 0.3408471 , 1.04645729, 0.72643334],
       [0.1980827 , 1.57364726, 0.39035979, 1.67173135, 0.9210825 ],
       [0.52586645, 0.60021836, 0.99369019, 0.9282068 , 0.94634491],
       [0.72692764, 0.94875091, 0.86374229, 1.09063315, 0.00412456],
       [0.93076301, 0.3343119 , 0.

In [167]:
baseadd = (b+randadd)
baseadd

array([[0.67099383, 0.9082271 , 1.30863311, 0.13836125, 0.37077377],
       [0.80051552, 0.2554239 , 1.71989836, 0.57499728, 0.58092856],
       [0.55595365, 0.37022075, 0.31031638, 0.04638312, 0.89031461],
       [0.4462557 , 0.48636655, 0.49408568, 0.43520371, 0.26299108],
       [1.23403999, 0.98933346, 1.2675608 , 0.96707127, 0.24639843],
       [0.7003624 , 0.72404507, 1.35443244, 0.82925964, 0.50561157],
       [1.34918852, 0.91351048, 0.22866098, 0.52384588, 1.05836764],
       [0.09020234, 0.62528357, 1.7359527 , 0.04652401, 0.40306654],
       [0.8626351 , 0.16699363, 1.92609549, 0.85600583, 1.23783086],
       [0.19067204, 1.7169733 , 1.10917549, 0.35402994, 0.49338102],
       [0.22329962, 1.49743681, 0.34084711, 1.04645727, 0.72643334],
       [0.1980827 , 1.57364719, 0.39035979, 1.67173138, 0.92108251],
       [0.52586642, 0.60021833, 0.9936902 , 0.92820681, 0.94634493],
       [0.72692763, 0.94875092, 0.86374228, 1.09063322, 0.00412456],
       [0.93076297, 0.33431191, 0.

In [168]:
adenseadd - baseadd

array([[-2.44804763e-08, -1.17692616e-08, -2.18917533e-08,
        -4.33389269e-09, -5.06627373e-09],
       [ 1.00584422e-08,  4.06022549e-10, -2.12841382e-08,
         2.71126038e-08,  8.77549633e-09],
       [-2.42979592e-08,  1.04042330e-10,  4.47188830e-09,
        -4.05685596e-10,  2.49759788e-08],
       [ 1.25616724e-08, -6.48339371e-09, -1.48251075e-08,
        -4.24316893e-09, -9.11035314e-09],
       [ 3.51235077e-08, -1.35460703e-08, -7.86857695e-08,
         2.57611834e-08,  6.33626807e-09],
       [-1.29658345e-08,  2.86361911e-08,  2.84109429e-08,
        -4.36689174e-09, -2.69647490e-08],
       [ 4.32894762e-08,  2.28349046e-08,  2.17468077e-09,
        -2.83249951e-08, -2.81688219e-08],
       [ 3.55165186e-09,  3.25458355e-08, -8.20960091e-08,
        -6.18497253e-10,  1.03582808e-08],
       [-2.73272084e-08,  5.93349325e-09, -2.39009745e-09,
         1.50987324e-08,  2.03117063e-08],
       [ 2.23531349e-09,  5.47390266e-09, -4.15198316e-08,
        -1.39485469e-08

In [169]:
(adenseadd - baseadd).sum()

-1.1252679066320326e-07

### Update Vals (add sparset)

In [170]:
multt=np.zeros(mult.T.shape)

for row in range(multt.shape[0]):
    for col in range(multt.shape[1]):
        multt[row][col] = mult[col][row]

In [171]:
multdata, multcols, multnnz, multellw = to_data(multt)
multdata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=multdata)
multcols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=multcols)
multnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=multnnz)

In [172]:
a.shape, b.shape

((8, 16), (16, 5))

In [173]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [174]:
rows = mult.T.shape[0]

In [175]:
mult = mult.astype(np.float32)

In [176]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)
knl = prg.adddense  # Use this Kernel object for repeated calls
knl(queue, [rows], None, multdata_buf, multcols_buf, multnnzs_buf, np.float32(1), np.uint32(multellw), 
    sdatat_buf, sidxst_buf, snnzst_buf, np.uint32(topk))

NameError: name 'sdatat_buf' is not defined

In [ ]:
mult.T

In [ ]:
data_res = np.empty_like(multdata)
cols_res = np.empty_like(multcols)
nnzs_res = np.empty_like(multnnz)
cl.enqueue_copy(queue, data_res, multdata_buf, is_blocking=True)
cl.enqueue_copy(queue, cols_res, multcols_buf, is_blocking=True)
cl.enqueue_copy(queue, nnzs_res, multnnzs_buf, is_blocking=True)

In [ ]:
multt-data_res.reshape(multt.shape)

In [ ]:
nnzs_res

In [ ]:
adenseaddt = to_dense(data_res, cols_res, nnzs_res, multellw, multt.shape)
adenseaddt

In [ ]:
multt-adenseaddt

In [ ]:
adenseaddt

In [ ]:
adenseadd.T == adenseaddt

### Update Vals (add topk to sparse)

In [ ]:
matadd = np.random.randn(*a.shape).astype(np.float32)
matadd

In [ ]:
a

In [ ]:
a_added = a + matadd

In [ ]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
add_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=matadd)

prg = cl.Program(ctx, """
    // Every global_id_0 works on a row
    __kernel void adddense(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            float  lr,
                            uint   ellwidth,
                            uint   awidth,
                            __global  float* vector_x    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);

      uint nnz    = rowNnz[gid];
      uint baseidxs = gid*ellwidth;
      uint baseidxd = gid*awidth;
      
      for (uint i=0; i<awidth; i++) {
        float addval = vector_x[baseidxd+i];
        //if (gid==1)
        //  printf("\\nADD VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[baseidxs+i]);
        if (addval == 0) {
          continue;
        }
        if (i == colIdx[baseidxs+i]) {
          matData[baseidxs+i] += addval;
        } else {
          if (rowNnz[gid] >= ellwidth) {
            break;
          }
          if (i > colIdx[baseidxs+i]) {
            rowNnz[gid] += 1;
            //if (gid==1)
            //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
            matData[baseidxs+i] = addval;
            colIdx[baseidxs+i] = i;
            continue;
          }
          for (uint j=nnz; j>i; j--) {
            //printf("\\nMOVE:%.2f", matData[baseidx+j-1]);
            colIdx[baseidxs+j] = colIdx[baseidxs+j-1];
            matData[baseidxs+j] = matData[baseidxs+j-1];
          }
          rowNnz[gid] += 1;
          nnz = rowNnz[gid];
          //if (gid==1)
          //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i] = addval;
          colIdx[baseidxs+i] = i;
          if (nnz >= ellwidth)
            break;
        }
      }
    }""").build()

In [ ]:
a.shape, b.shape

In [ ]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [ ]:
rows = a.shape[0]

In [ ]:
mult = mult.astype(np.float32)

In [177]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)
knl = prg.adddense  # Use this Kernel object for repeated calls
knl(queue, [rows], None, adata_buf, acols_buf, annzs_buf, np.float32(1), np.uint32(ellwa),np.uint32(a.shape[1]), add_buf)

NameError: name 'add_buf' is not defined

In [ ]:
matadd[0][0]

In [ ]:
data_res = np.empty_like(adata)
cols_res = np.empty_like(acols)
nnzs_res = np.empty_like(annz)
cl.enqueue_copy(queue, data_res, adata_buf)
cl.enqueue_copy(queue, cols_res, acols_buf)
cl.enqueue_copy(queue, nnzs_res, annzs_buf)

In [ ]:
adenseadd = to_dense(data_res, cols_res, nnzs_res, ellwa, a.shape)
adenseadd

In [ ]:
a

In [ ]:
matadd

In [178]:
a_added

NameError: name 'a_added' is not defined

In [ ]:
adenseadd

In [ ]:
adenseadd == a_added

### update vals

In [179]:
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
add_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=matadd)

prg = cl.Program(ctx, """
    // Every global_id_0 works on a row
    __kernel void adddenset(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            float  lr,
                            uint   ellwidth,
                            uint   aheight,
                            __global  float* vector_x    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint ncols = get_global_size(0);

      uint nnz    = rowNnz[gid];
      uint baseidxs = gid*ellwidth;
      
      for (uint i=0; i<aheight; i++) {
        if (nnz > ellwidth)
            break;
        uint baseidxd = i*ncols+gid;
        float addval = vector_x[baseidxd];
        //if (gid==1)
        //  printf("\\nADD VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[baseidxs+i]);
        if (addval == 0) {
          continue;
        }
        if (i == colIdx[baseidxs+i]) {
          printf("\\nADD VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i] += addval;
        } else {
          if (rowNnz[gid] >= ellwidth) {
            break;
          }
          if (i > colIdx[baseidxs+i]) {
            rowNnz[gid] += 1;
            //if (gid==1)
            //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
            matData[baseidxs+i] = addval;
            colIdx[baseidxs+i] = i;
            continue;
          }
          for (uint j=nnz; j>i; j--) {
            //printf("\\nMOVE:%.2f", matData[baseidx+j-1]);
            colIdx[baseidxs+j] = colIdx[baseidxs+j-1];
            matData[baseidxs+j] = matData[baseidxs+j-1];
          }
          rowNnz[gid] += 1;
          nnz = rowNnz[gid];
          //if (gid==1)
          //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i] = addval;
          colIdx[baseidxs+i] = i;
        }
      }
    }""").build()

NameError: name 'matadd' is not defined

In [ ]:
a.shape, b.shape

In [ ]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [ ]:
cols = a.shape[1]

In [180]:
mult = mult.astype(np.float32)

In [181]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)
knl = prg.adddenset  # Use this Kernel object for repeated calls
knl(queue, [cols], None, adatat_buf, acolst_buf, annzst_buf, np.float32(1), np.uint32(ellwat),np.uint32(a.T.shape[1]), add_buf)

AttributeError: 'adddenset' was not found as a program info attribute or as a kernel name

In [ ]:
matadd[0][0]

In [ ]:
datat_res = np.empty_like(adatat)
colst_res = np.empty_like(acolst)
nnzst_res = np.empty_like(annzt)
cl.enqueue_copy(queue, datat_res, adatat_buf)
cl.enqueue_copy(queue, colst_res, acolst_buf)
cl.enqueue_copy(queue, nnzst_res, annzst_buf)

In [ ]:
adenseaddt = to_dense(datat_res, colst_res, nnzst_res, ellwat, a.T.shape).T
adenseaddt

In [182]:
a

array([[0.        , 0.68347037, 0.8035886 , 0.        , 0.        ,
        0.        , 0.        , 0.08023349, 0.8858316 , 0.        ,
        0.17861794, 0.        , 0.46201044, 0.        , 0.        ,
        0.        ],
       [0.9562663 , 0.5531271 , 0.        , 0.        , 0.5375557 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.20370705, 0.35530138, 0.        , 0.        ,
        0.9186601 ],
       [0.        , 0.20103766, 0.        , 0.        , 0.        ,
        0.74341315, 0.        , 0.        , 0.        , 0.5787496 ,
        0.        , 0.05387774, 0.        , 0.5917517 , 0.17797045,
        0.        ],
       [0.        , 0.        , 0.        , 0.14296253, 0.9103574 ,
        0.03242496, 0.        , 0.71459514, 0.        , 0.        ,
        0.        , 0.        , 0.7448136 , 0.        , 0.        ,
        0.66635793],
       [0.        , 0.        , 0.        , 0.59893256, 0.        ,
        0.        , 0.        , 

In [183]:
matadd

NameError: name 'matadd' is not defined

In [ ]:
a_added

In [ ]:
adenseaddt == a_added

### Make Random

In [184]:
rand = SparseTensor.uniform(2,4)
rand

<SparseTensor <GPUBuffer with shape (8,)> with grad None>

In [185]:
rand.to_numpy()

array([[6.40131111e-05, 5.51252815e-05, 9.75026924e-05, 1.21387111e-05],
       [4.27303312e-05, 6.81816964e-05, 3.21073094e-05, 9.09540831e-05]])

In [186]:
rand.data

<GPUBuffer with shape (8,)>

### update vals

In [187]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)

In [188]:
prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         float lr,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,
                         __global  uint* updateyidx
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint gid2 = get_global_id(1);
      uint topk = get_global_size(0);
      uint bs = get_global_size(1);
      uint baseupdateidx = topk*topk*gid2;
      uint baseidxidx = topk*gid2;
      uint col = updateyidx[baseidxidx+gid];

      for (uint i=0; i<topk; i++) {
        float val = updatevals[baseupdateidx+gid*topk+i];
        uint row = updatexidx[baseidxidx+i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = row*ellwidth+i;
          if (colIdx[idx] >= col) {
            //printf("\\nFOUND:%i/%i  - idx:%i", colIdx[idx], col, idx);
            if (colIdx[idx] == col) {
              matData[idx] += -val*lr;
              printf("\\nUPDATE[%i,%i]: %f", row,col, val);
              break;
            } else {
              // insert new column
              printf("\\nINSERT[%i,%i]: %.2f", row,col, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = row*ellwidth+j;
                matData[idx2] = matData[idx2-1];
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = -val*lr;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }
    }""").build()

In [189]:
prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         float lr,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,
                         __global  uint* updateyidx
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint topk = get_global_size(0);
      uint col = updateyidx[gid];

      for (uint i=0; i<topk; i++) {
        float val = updatevals[baseupdateidx+gid*topk+i];
        uint row = updatexidx[baseidxidx+i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = row*ellwidth+i;
          if (colIdx[idx] >= col) {
            //printf("\\nFOUND:%i/%i  - idx:%i", colIdx[idx], col, idx);
            if (colIdx[idx] == col) {
              matData[idx] += -val*lr;
              printf("\\nUPDATE[%i,%i]: %f", row,col, val);
              break;
            } else {
              // insert new column
              printf("\\nINSERT[%i,%i]: %.2f", row,col, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = row*ellwidth+j;
                matData[idx2] = matData[idx2-1];m
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = -val*lr;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }
    }""").build()

RuntimeError: clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE

Build on <pyopencl.Device 'GeForce GTX 1080 Ti' on 'NVIDIA CUDA' at 0x55ec5d70cbf0>:

<kernel>:17:32: error: use of undeclared identifier 'baseupdateidx'
        float val = updatevals[baseupdateidx+gid*topk+i];
                               ^
<kernel>:18:31: error: use of undeclared identifier 'baseidxidx'
        uint row = updatexidx[baseidxidx+i];
                              ^
<kernel>:32:49: error: use of undeclared identifier 'm'
                matData[idx2] = matData[idx2-1];m
                                                ^

(options: -I /home/fpaboim/.conda/envs/tinygrad/lib/python3.8/site-packages/pyopencl/cl)
(source saved as /tmp/tmpfvsfx1gp.cl)

In [ ]:
knl = prg.addvals  # Use this Kernel object for repeated calls
knl(queue, [topk,1], None, adata_buf, acols_buf, annzs_buf, np.float32(1), np.uint32(ellwa), x_cp_buf, x_idx_buf, y_idx_buf)

resa = np.empty_like(adata)
resaidx = np.zeros(acols.shape).astype(np.uint32)
resannz = np.zeros(annz.shape).astype(np.uint32)

cl.enqueue_copy(queue, resa, adata_buf)
cl.enqueue_copy(queue, resaidx, acols_buf)
cl.enqueue_copy(queue, resannz, annzs_buf)

In [ ]:
resa.shape, resaidx.shape, resannz.shape, ellwa, a.T.shape

In [ ]:
adenseadd = to_dense(resa, resaidx, resannz, ellwa, a.shape)
adenseadd

In [ ]:
adenseadd - adense

In [ ]:
adenseadd == adense

In [ ]:
ellwa

In [ ]:
adata2 = adata.reshape(-1, ellwa)
adata2

In [ ]:
resa = resa.reshape(-1, ellwa)
resa

In [ ]:
resa - adata2

In [190]:
acols

array([ 1,  2,  7,  8, 10, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        1,  4, 11, 12, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,
        9, 11, 13, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  4,  5,
        7, 12, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  7,  9, 11,
       14, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,  7, 10,
       11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  3,  5, 12, 13,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  6,  8, 11, 15,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=uint32)

In [191]:
resaidx

NameError: name 'resaidx' is not defined

In [ ]:
resannz

In [ ]:
annz

### update vals2

In [ ]:
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)

In [ ]:
prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         float lr,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,
                         __global  uint* updateyidx
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint gid2 = get_global_id(1);
      uint topk = get_global_size(0);
      uint bs = get_global_size(1);
      uint baseupdateidx = topk*topk*gid2;
      uint baseidxidx = topk*gid2;
      uint row = updateyidx[baseidxidx+gid];

      for (uint i=0; i<topk; i++) {
        float val = updatevals[baseupdateidx+gid*topk+i];
        uint col = updatexidx[baseidxidx+i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = row*ellwidth+i;
          if (colIdx[idx] >= col) {
            //printf("\\nFOUND:%i/%i  - idx:%i", colIdx[idx], col, idx);
            if (colIdx[idx] == col) {
              matData[idx] += -val*lr;
              printf("\\nUPDATE[%i,%i]: %f", row,col, val);
              break;
            } else {
              // insert new column
              printf("\\nINSERT[%i,%i]: %.2f", row,col, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = row*ellwidth+j;
                matData[idx2] = matData[idx2-1];
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = -val*lr;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }
    }""").build()

In [192]:
knl = prg.addvals  # Use this Kernel object for repeated calls
knl(queue, [topk,bs], None, adatat_buf, acolst_buf, annzst_buf, np.float32(1), np.uint32(ellwat), x_cp_buf, x_idx_buf, y_idx_buf)

resat = np.empty_like(adatat)
resaidxt = np.zeros(acolst.shape).astype(np.uint32)
resannzt = np.zeros(annzt.shape).astype(np.uint32)

cl.enqueue_copy(queue, resat, adatat_buf)
cl.enqueue_copy(queue, resaidxt, acolst_buf)
cl.enqueue_copy(queue, resannzt, annzst_buf)


UPDATE[0,1]: 0.155507
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.00
INSERT[0,0]: 0.70
UPDATE[0,0]: 0.000000
UPDATE[0,0]: 0.000000
UPDATE[0,0]: 0.000000
UPDATE[0,0]: 0.000000
UPDATE[0,0]: 0.000000
UPDATE[0,0]: 0.000000
UPDATE[0,0]: 0.000000
UPDATE[0,0]: 0.000000
UPDATE[0,0]: 0.000000
UPDATE[0,0]: 0.000000
UPDATE[0,0]: 0.000000
UPDATE[0,0]: 0.000000
UPDATE[0,0]: 0.000000
UPDATE[0,0]: 0.000000
UPDATE[0,0]: 0.000000
UPDATE[1,2]: 1.416093
INSERT[1,1]: 0.19
INSERT[1,3]: 0.30
INSERT[1,4]: 0.30
UPDATE[3,3]: 0.320511
INSERT[3,1]: 0.31
INSERT[3,2]: 1.20
UPDATE[3,4]: 0.631148
UPDATE[4,1]: 0.967791
UPDATE[4,3]: 0.135177
INSERT[4,2]: 0.82
UPDATE[7,3]: 0.096748
INSERT[7,1]: 0.28
INSERT[7,2]

: 0.99
UPDATE[7,4]: 0.839095

In [193]:
ellwa

16

In [194]:
resat.shape, resaidxt.shape, resannzt.shape

((128,), (128,), (16,))

In [195]:
adenseaddt = to_dense(resat, resaidxt, resannzt, ellwat, a.T.shape)
adenseaddt

array([[ 0.        ,  0.80075902,  0.        ,  0.        ,  0.        ,
         0.11415514,  0.28872353,  0.        ],
       [ 0.68347037, -0.18735047, -1.21505558,  0.        , -0.29892322,
         0.82574075,  0.04889954,  0.        ],
       [ 0.80358863,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -1.19507909, -0.17754884, -0.03221506,
         0.        ,  0.6493426 ,  0.35563806],
       [ 0.        , -0.43023491, -0.81877571,  0.77518052,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.74341315,  0.03242496,  0.        ,
         0.27978534,  0.33329687,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.69630474],
       [ 0.08023349,  0.        , -0.99110806,  0.6178472 ,  0.08260983,
         0.25018668,  0.        ,  0.        ],
       [ 0.88583159,  0.        

In [196]:
adenseadd == adenseaddt.T

/tmp/ipykernel_31620/1679493034.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  adenseadd == adenseaddt.T


False

In [197]:
adata2t = adatat.reshape(-1, ellwat)
adata2t

array([[0.9562663 , 0.11415514, 0.28872353, 0.67626965, 0.        ,
        0.        , 0.        , 0.        ],
       [0.68347037, 0.5531271 , 0.20103766, 0.82574075, 0.04889954,
        0.        , 0.        , 0.        ],
       [0.8035886 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.14296253, 0.59893256, 0.6493426 , 0.35563806, 0.        ,
        0.        , 0.        , 0.        ],
       [0.5375557 , 0.9103574 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.74341315, 0.03242496, 0.27978534, 0.33329687, 0.        ,
        0.        , 0.        , 0.        ],
       [0.69630474, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.08023349, 0.71459514, 0.9217044 , 0.25018668, 0.        ,
        0.        , 0.        , 0.        ],
       [0.8858316 , 0.6753613 , 0.        , 0.        , 0.        ,
        0.        , 0.      

In [198]:
resat = resat.reshape(-1, ellwat)
resat

array([[-0.6996368 , -0.        , -0.        , -0.        , -0.        ,
         0.800759  ,  0.11415514,  0.28872353],
       [ 0.        ,  0.68347037, -0.18735047, -1.2150556 , -0.29892322,
         0.82574075,  0.04889954,  0.        ],
       [ 0.8035886 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [-1.1950791 , -0.17754884, -0.03221506,  0.6493426 ,  0.35563806,
         0.        ,  0.        ,  0.        ],
       [-0.4302349 , -0.8187757 ,  0.7751805 ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.74341315,  0.03242496,  0.27978534,  0.33329687,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.69630474,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.08023349, -0.99110806,  0.6178472 ,  0.08260983,  0.25018668,
         0.        ,  0.        ,  0.        ],
       [ 0.8858316 ,  0.6753613 

In [199]:
resat - adata2t

array([[-1.6559031 , -0.11415514, -0.28872353, -0.67626965, -0.        ,
         0.800759  ,  0.11415514,  0.28872353],
       [-0.68347037,  0.13034326, -0.38838813, -2.0407963 , -0.34782276,
         0.82574075,  0.04889954,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [-1.3380417 , -0.7764814 , -0.68155766,  0.29370454,  0.35563806,
         0.        ,  0.        ,  0.        ],
       [-0.9677906 , -1.7291331 ,  0.7751805 ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        , -1.7057033 , -0.3038572 , -0.16757685,  0.25018668,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        

In [200]:
acols

array([ 1,  2,  7,  8, 10, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        1,  4, 11, 12, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,
        9, 11, 13, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  4,  5,
        7, 12, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  7,  9, 11,
       14, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,  7, 10,
       11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  3,  5, 12, 13,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  6,  8, 11, 15,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=uint32)

In [201]:
resaidx

NameError: name 'resaidx' is not defined

In [ ]:
resannz

In [ ]:
annz

# OTHER

import numpy as np
import pyopencl as cl

mf = cl.mem_flags

dim = 16
topk = 4

x = np.random.rand(dim).astype(np.float32)
y = np.random.rand(dim).astype(np.float32)
x.shape,y.shape

dim1 = 4
dim2 = 8
dim3 = 1

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx,
        properties=cl.command_queue_properties.PROFILING_ENABLE)

sparsity = 0.2

a = np.zeros((dim1,dim2))
b = np.random.rand(dim2,dim3).flatten().astype(np.float32)

a.shape, b.shape

In [ ]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
val_out_buf = cl.Buffer(ctx, mf.READ_WRITE, 4*topk*topk)
x_idx_buf = cl.Buffer(ctx, mf.READ_WRITE, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.READ_WRITE, topk*4)

prg = cl.Program(ctx, """
// Every global_id_0 works on a row
__kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                         __global  float* y,    // INPUT
                         __global  float* xout,    // INPUT
                         uint topk,
                         __global  uint* xoutidx,    // INPUT
                         __global  uint* youtidx    // INPUT
                        ) { // LOCAL SHARED BUFFER
  uint gid = get_global_id(0);
  uint n = get_global_size(0);
  
  xout[gid] = x[gid];
  xoutidx[gid] = gid;
  youtidx[gid] = gid;
  
  float valx = x[gid];
  float valy = y[gid];
  uint posx = 0;
  uint posy = 0;
  for (uint i = 0; i < n; i++) {
    float tempval = x[i];
    float tempval2 = y[i];
    bool larger = tempval > valx;
    bool larger2 = tempval2 > valy;
      
    posx += (larger)?1:0;
    posy += (larger2)?1:0;
  }
  //printf("posx:%i", posx);
  if (posx < topk) {
    xoutidx[posx] = gid;
  }
  if (posy < topk) {
    youtidx[posy] = gid;
  }
  if (gid < topk) {
    uint i = gid;
    for (uint j=0; j<topk; j++) {
      xout[gid*topk+j] = x[xoutidx[gid]] * y[youtidx[j]];
    }
  }
}""").build()

In [ ]:
knl = prg.genwupdate2  # Use this Kernel object for repeated calls
event = knl(queue, [dim,], None, x_buf, y_buf, val_out_buf, np.uint32(topk), x_idx_buf, y_idx_buf)

#event.wait()
val_out = np.zeros(topk*topk).astype(np.float32)
resxidx = np.zeros(topk).astype(np.uint32)
resyidx = np.zeros(topk).astype(np.uint32)

cl.enqueue_copy(queue, val_out, val_out_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf, wait_for=[event])
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [ ]:
val_out

In [ ]:
resxidx

In [ ]:
resyidx

In [ ]:
asdf

In [ ]:
from __future__ import division

KERNEL_CODE = """
// Thread block size
#define BLOCK_SIZE %(block_size)d
// Matrix dimensions
// (chosen as multiples of the thread block size for simplicity)
#define WA %(w_a)d // Matrix A width
#define HA %(h_a)d // Matrix A height
#define WB %(w_b)d // Matrix B width
#define HB WA  // Matrix B height
#define WC WB  // Matrix C width
#define HC HA  // Matrix C height
/*
 * Copyright 1993-2009 NVIDIA Corporation.  All rights reserved.
 *
 * NVIDIA Corporation and its licensors retain all intellectual property and
 * proprietary rights in and to this software and related documentation.
 * Any use, reproduction, disclosure, or distribution of this software
 * and related documentation without an express license agreement from
 * NVIDIA Corporation is strictly prohibited.
 *
 * Please refer to the applicable NVIDIA end user license agreement (EULA)
 * associated with this source code for terms and conditions that govern
 * your use of this NVIDIA software.
 *
 */
/* Matrix multiplication: C = A * B.
 * Device code.
 */
#define AS(j, i) As[i + j * BLOCK_SIZE]
#define BS(j, i) Bs[i + j * BLOCK_SIZE]
////////////////////////////////////////////////////////////////////////////////
//! Matrix multiplication on the device: C = A * B
//! WA is A's width and WB is B's width
////////////////////////////////////////////////////////////////////////////////
__kernel __attribute__((reqd_work_group_size(16,16,1))) 
void
matrixMul( __global float* C, __global float* A, __global float* B)
{
    __local float As[BLOCK_SIZE*BLOCK_SIZE];
    __local float Bs[BLOCK_SIZE*BLOCK_SIZE];
    // Block index
    int bx = get_group_id(0);
    int by = get_group_id(1);
    // Thread index
    int tx = get_local_id(0);
    int ty = get_local_id(1);
    // Index of the first sub-matrix of A processed by the block
    int aBegin = WA * BLOCK_SIZE * by;
    // Index of the last sub-matrix of A processed by the block
    int aEnd   = aBegin + WA - 1;
    // Step size used to iterate through the sub-matrices of A
    int aStep  = BLOCK_SIZE;
    // Index of the first sub-matrix of B processed by the block
    int bBegin = BLOCK_SIZE * bx;
    // Step size used to iterate through the sub-matrices of B
    int bStep  = BLOCK_SIZE * WB;
    // Csub is used to store the element of the block sub-matrix
    // that is computed by the thread
    float Csub = 0.0f;
    // Loop over all the sub-matrices of A and B
    // required to compute the block sub-matrix
    for (int a = aBegin, b = bBegin;
             a <= aEnd;
             a += aStep, b += bStep) {
        // Load the matrices from device memory
        // to shared memory; each thread loads
        // one element of each matrix
        AS(ty, tx) = A[a + WA * ty + tx];
        BS(ty, tx) = B[b + WB * ty + tx];
        // Synchronize to make sure the matrices are loaded
        barrier(CLK_LOCAL_MEM_FENCE);
        // Multiply the two matrices together;
        // each thread computes one element
        // of the block sub-matrix
        for (int k = 0; k < BLOCK_SIZE; ++k)
            Csub += AS(ty, k) * BS(k, tx);
        // Synchronize to make sure that the preceding
        // computation is done before loading two new
        // sub-matrices of A and B in the next iteration
        barrier(CLK_LOCAL_MEM_FENCE);
    }
    // Write the block sub-matrix to device memory;
    // each thread writes one element
    C[get_global_id(1) * get_global_size(0) + get_global_id(0)] = Csub;
}
"""


In [ ]:
a2 = np.random.rand(4,4)

In [202]:
a2

NameError: name 'a2' is not defined

In [ ]:
a2.sum(axis=1)

In [ ]:
b2 = np.random.rand(4,4)

In [ ]:
b2

In [203]:
b2.sum(axis=0)

NameError: name 'b2' is not defined

In [ ]:
matmul = a2.dot(b2)
matmul